# Facial Expression Recognition with Multi-Task Learning

This notebook implements a comprehensive facial expression recognition system using multi-task learning to predict both discrete emotions (8 classes) and continuous valence/arousal values. We'll compare multiple state-of-the-art CNN architectures:

- **VGG16** (baseline/classic architecture)
- **EfficientNet-B0/B2** (efficient scaling)
- **ConvNeXt-Tiny** (modern CNN with transformer-like design)

## Dataset Overview

The dataset contains facial images with annotations for:
- **Expression labels**: 8 emotion classes (0-7)
- **Valence**: Continuous values from -1 to 1 (negative to positive emotions)
- **Arousal**: Continuous values from -1 to 1 (calm to excited)
- **Landmarks**: 68 facial landmark points (optional for this assignment)

## Evaluation Metrics

### Classification Metrics:
- Accuracy, F1-macro, Cohen's κ, ROC-AUC, PR-AUC, Krippendorff's α

### Regression Metrics:
- RMSE, Pearson correlation, SAGR, Concordance Correlation Coefficient (CCC)

## 0. Environment & Paths

Setting up the environment with required packages, paths, and configurations.

In [ ]:
# Install required packages
!pip install timm torchmetrics scikit-learn matplotlib seaborn tqdm pillow
!pip install krippendorff  # For Krippendorff's alpha

# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
import timm

# Sklearn for metrics and preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (accuracy_score, f1_score, cohen_kappa_score, 
                           roc_auc_score, average_precision_score, 
                           confusion_matrix, classification_report,
                           mean_squared_error, r2_score)
from sklearn.preprocessing import LabelBinarizer

# Other utilities
from PIL import Image
from tqdm import tqdm
import json
import random
import time
from scipy.stats import pearsonr
import krippendorff

print("All packages imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")
    print(f"CUDA version: {torch.version.cuda}")

In [ ]:
# Set random seeds for reproducibility
SEED = 4
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Set to True for performance if input sizes are consistent

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Data paths
DATA_DIR = Path('/kaggle/input/dataset/Dataset')
IMAGES_DIR = DATA_DIR / 'images'
ANNOTATIONS_DIR = DATA_DIR / 'annotations'

# Create output directories
ARTIFACTS_DIR = Path('./artifacts')
FIGURES_DIR = Path('./figures')
ARTIFACTS_DIR.mkdir(exist_ok=True)
FIGURES_DIR.mkdir(exist_ok=True)

print(f"Data directory: {DATA_DIR}")
print(f"Images directory: {IMAGES_DIR}")
print(f"Annotations directory: {ANNOTATIONS_DIR}")
print(f"Images directory exists: {IMAGES_DIR.exists()}")
print(f"Annotations directory exists: {ANNOTATIONS_DIR.exists()}")

# Expression labels mapping (assuming 8 basic emotions)
EMOTION_LABELS = {
    0: 'Neutral',
    1: 'Happy', 
    2: 'Sad',
    3: 'Angry',
    4: 'Fearful',
    5: 'Disgusted',
    6: 'Surprised',
    7: 'Contemptuous'
}

print(f"Emotion labels: {EMOTION_LABELS}")

## 1. Read Dataset & Build Index

Loading all annotation files and building a comprehensive DataFrame with image paths and all target variables.

In [ ]:
def load_annotations(annotations_dir, images_dir):
    """
    Load all annotation files and build a comprehensive DataFrame
    """
    data_list = []
    
    # Get all annotation files
    annotation_files = list(annotations_dir.glob('*.npy'))
    
    # Group files by image ID
    image_ids = set()
    for f in annotation_files:
        # Extract image ID (before the first underscore)
        image_id = f.stem.split('_')[0]
        image_ids.add(image_id)
    
    print(f"Found {len(image_ids)} unique images")
    print(f"Found {len(annotation_files)} annotation files")
    
    for image_id in tqdm(sorted(image_ids, key=int), desc="Loading annotations"):
        # Construct file paths
        exp_file = annotations_dir / f"{image_id}_exp.npy"
        val_file = annotations_dir / f"{image_id}_val.npy"
        aro_file = annotations_dir / f"{image_id}_aro.npy"
        lnd_file = annotations_dir / f"{image_id}_lnd.npy"
        
        # Construct image path
        image_path = images_dir / f"{image_id}.jpg"
        
        # Check if image file exists
        if not image_path.exists():
            continue
            
        # Load annotations if they exist
        try:
            expression = np.load(exp_file) if exp_file.exists() else None
            valence = np.load(val_file) if val_file.exists() else None
            arousal = np.load(aro_file) if aro_file.exists() else None
            landmarks = np.load(lnd_file) if lnd_file.exists() else None
            
            # Skip if essential annotations are missing
            if expression is None or valence is None or arousal is None:
                continue
                
            data_list.append({
                'image_id': int(image_id),
                'image_path': str(image_path),
                'expression': int(expression),
                'valence': float(valence),
                'arousal': float(arousal),
                'landmarks': landmarks.tolist() if landmarks is not None else None
            })
            
        except Exception as e:
            print(f"Error loading annotations for image {image_id}: {e}")
            continue
    
    df = pd.DataFrame(data_list)
    print(f"Successfully loaded {len(df)} samples")
    
    return df

# Load the dataset
print("Loading dataset annotations...")
df = load_annotations(ANNOTATIONS_DIR, IMAGES_DIR)

print(f"\nDataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

print(f"\nBasic statistics:")
print(df.describe())

In [ ]:
# Analyze the dataset
print("Dataset Analysis:")
print("="*50)

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

# Expression distribution
print("\nExpression distribution:")
expr_counts = df['expression'].value_counts().sort_index()
for expr, count in expr_counts.items():
    emotion_name = EMOTION_LABELS.get(expr, f'Unknown_{expr}')
    print(f"  {expr} ({emotion_name}): {count} samples ({count/len(df)*100:.1f}%)")

# Valence and arousal statistics
print(f"\nValence range: [{df['valence'].min():.3f}, {df['valence'].max():.3f}]")
print(f"Arousal range: [{df['arousal'].min():.3f}, {df['arousal'].max():.3f}]")

# Check for uncertain annotations (valence/arousal == -2)
uncertain_val = (df['valence'] == -2).sum()
uncertain_aro = (df['arousal'] == -2).sum()
print(f"\nUncertain annotations:")
print(f"  Valence (-2): {uncertain_val} samples ({uncertain_val/len(df)*100:.1f}%)")
print(f"  Arousal (-2): {uncertain_aro} samples ({uncertain_aro/len(df)*100:.1f}%)")

# Visualize distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Expression distribution
axes[0, 0].bar(range(len(expr_counts)), expr_counts.values)
axes[0, 0].set_xlabel('Expression Label')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Expression Distribution')
axes[0, 0].set_xticks(range(len(expr_counts)))
axes[0, 0].set_xticklabels([EMOTION_LABELS.get(i, f'Unk_{i}') for i in expr_counts.index], rotation=45)

# Valence distribution
axes[0, 1].hist(df['valence'], bins=30, alpha=0.7, edgecolor='black')
axes[0, 1].set_xlabel('Valence')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_title('Valence Distribution')
axes[0, 1].axvline(x=0, color='red', linestyle='--', alpha=0.7)

# Arousal distribution
axes[1, 0].hist(df['arousal'], bins=30, alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Arousal')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Arousal Distribution')
axes[1, 0].axvline(x=0, color='red', linestyle='--', alpha=0.7)

# Valence-Arousal scatter
scatter = axes[1, 1].scatter(df['valence'], df['arousal'], c=df['expression'], cmap='tab10', alpha=0.6, s=20)
axes[1, 1].set_xlabel('Valence')
axes[1, 1].set_ylabel('Arousal')
axes[1, 1].set_title('Valence-Arousal Space (colored by expression)')
axes[1, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[1, 1].axvline(x=0, color='black', linestyle='-', alpha=0.3)
plt.colorbar(scatter, ax=axes[1, 1])

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'dataset_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nDataset loaded successfully! Shape: {df.shape}")

## 2. Train/Val/Test Split

Cleaning the dataset and creating stratified splits while preserving class distributions.

In [ ]:
# Data cleaning and preprocessing
print("Data Preprocessing:")
print("="*50)

# Create a copy for processing
df_clean = df.copy()

print(f"Original dataset size: {len(df_clean)}")

# Filter out uncertain annotations for regression targets (valence/arousal == -2)
# We'll keep these samples for classification but mark them for regression exclusion
df_clean['valid_for_regression'] = ((df_clean['valence'] != -2) & (df_clean['arousal'] != -2))

print(f"Samples valid for regression: {df_clean['valid_for_regression'].sum()}")
print(f"Samples valid only for classification: {(~df_clean['valid_for_regression']).sum()}")

# For regression, replace uncertain values with NaN (we'll handle this in the dataset class)
df_regression = df_clean.copy()
df_regression.loc[df_regression['valence'] == -2, 'valence'] = np.nan
df_regression.loc[df_regression['arousal'] == -2, 'arousal'] = np.nan

# Check class balance after filtering
print("\nClass distribution after cleaning:")
expr_counts = df_clean['expression'].value_counts().sort_index()
for expr, count in expr_counts.items():
    emotion_name = EMOTION_LABELS.get(expr, f'Unknown_{expr}')
    print(f"  {expr} ({emotion_name}): {count} samples ({count/len(df_clean)*100:.1f}%)")

# Check for minimum class size to ensure stratification is possible
min_class_count = expr_counts.min()
print(f"\nMinimum class count: {min_class_count}")

if min_class_count < 2:
    print("WARNING: Some classes have very few samples. Consider class merging or different split strategy.")

# Stratified train/val/test split
# First split: 80% train+val, 20% test
X = df_clean[['image_path', 'valence', 'arousal', 'valid_for_regression']].copy()
y = df_clean['expression'].copy()

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=SEED, 
    stratify=y
)

# Second split: 80% train, 20% val (of the remaining 80%)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, 
    test_size=0.25,  # 0.25 * 0.8 = 0.2 of total
    random_state=SEED, 
    stratify=y_trainval
)

print(f"\nSplit sizes:")
print(f"  Train: {len(X_train)} samples ({len(X_train)/len(df_clean)*100:.1f}%)")
print(f"  Val: {len(X_val)} samples ({len(X_val)/len(df_clean)*100:.1f}%)")
print(f"  Test: {len(X_test)} samples ({len(X_test)/len(df_clean)*100:.1f}%)")

# Verify stratification worked
print("\nClass distribution in splits:")
for split_name, y_split in [('Train', y_train), ('Val', y_val), ('Test', y_test)]:
    counts = y_split.value_counts().sort_index()
    print(f"\n{split_name}:")
    for expr, count in counts.items():
        emotion_name = EMOTION_LABELS.get(expr, f'Unknown_{expr}')
        print(f"  {expr} ({emotion_name}): {count} ({count/len(y_split)*100:.1f}%)")

# Create split DataFrames
train_df = pd.concat([X_train, y_train], axis=1)
train_df.columns = list(X_train.columns) + ['expression']

val_df = pd.concat([X_val, y_val], axis=1)
val_df.columns = list(X_val.columns) + ['expression']

test_df = pd.concat([X_test, y_test], axis=1)
test_df.columns = list(X_test.columns) + ['expression']

print(f"\nFinal split DataFrames created:")
print(f"  train_df: {train_df.shape}")
print(f"  val_df: {val_df.shape}")
print(f"  test_df: {test_df.shape}")

In [ ]:
# Save split CSVs for reproducibility
print("Saving split CSVs...")

# Add split information to original dataframe
df_with_splits = df_clean.copy()
df_with_splits['split'] = 'unknown'

# Set split labels
train_indices = train_df.index if hasattr(train_df, 'index') else X_train.index
val_indices = val_df.index if hasattr(val_df, 'index') else X_val.index
test_indices = test_df.index if hasattr(test_df, 'index') else X_test.index

df_with_splits.loc[train_indices, 'split'] = 'train'
df_with_splits.loc[val_indices, 'split'] = 'val' 
df_with_splits.loc[test_indices, 'split'] = 'test'

# Save individual split files
train_df_full = df_clean.loc[train_indices].copy()
val_df_full = df_clean.loc[val_indices].copy()
test_df_full = df_clean.loc[test_indices].copy()

train_df_full.to_csv(ARTIFACTS_DIR / 'train_split.csv', index=False)
val_df_full.to_csv(ARTIFACTS_DIR / 'val_split.csv', index=False)
test_df_full.to_csv(ARTIFACTS_DIR / 'test_split.csv', index=False)

# Save combined file with split information
df_with_splits.to_csv(ARTIFACTS_DIR / 'dataset_with_splits.csv', index=False)

print("Split CSVs saved to artifacts directory:")
print(f"  - train_split.csv: {len(train_df_full)} samples")
print(f"  - val_split.csv: {len(val_df_full)} samples") 
print(f"  - test_split.csv: {len(test_df_full)} samples")
print(f"  - dataset_with_splits.csv: {len(df_with_splits)} samples")

# Visualize split distributions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

splits = [('Train', train_df_full), ('Validation', val_df_full), ('Test', test_df_full)]
colors = ['skyblue', 'lightgreen', 'lightcoral']

for i, (split_name, split_df) in enumerate(splits):
    expr_counts = split_df['expression'].value_counts().sort_index()
    
    bars = axes[i].bar(range(len(expr_counts)), expr_counts.values, color=colors[i], alpha=0.8)
    axes[i].set_xlabel('Expression Label')
    axes[i].set_ylabel('Count')
    axes[i].set_title(f'{split_name} Set Distribution\n({len(split_df)} samples)')
    axes[i].set_xticks(range(len(expr_counts)))
    axes[i].set_xticklabels([EMOTION_LABELS.get(j, f'Unk_{j}') for j in expr_counts.index], rotation=45)
    
    # Add count labels on bars
    for bar, count in zip(bars, expr_counts.values):
        axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                    str(count), ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'split_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nData preprocessing and splitting completed successfully!")

## 3. Transforms

Defining data augmentation strategies for training and standard transforms for validation/testing.

In [ ]:
# Define transforms for different models
# ImageNet normalization values
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

def get_transforms(backbone_name='default', input_size=224):
    """
    Get appropriate transforms for different backbones
    """
    
    # Common base transforms
    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize first
        transforms.RandomResizedCrop(input_size, scale=(0.8, 1.0), ratio=(0.9, 1.1)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomRotation(degrees=10),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ToTensor(),
        transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
        # Removed lambda function that was causing pickle errors
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((input_size + 32, input_size + 32)),  # Slightly larger than target
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])
    
    test_transform = val_transform  # Same as validation
    
    return train_transform, val_transform, test_transform

# Create transforms for different models
transforms_dict = {
    'vgg16': get_transforms('vgg16', 224),
    'efficientnet_b0': get_transforms('efficientnet_b0', 224),
    'efficientnet_b2': get_transforms('efficientnet_b2', 260),  # EfficientNet-B2 uses 260x260
    'convnext_tiny': get_transforms('convnext_tiny', 224)
}

print("Transforms defined for all models:")
for model_name in transforms_dict.keys():
    print(f"  - {model_name}")

# Visualize some augmented examples
def visualize_transforms(df, transforms_tuple, num_samples=8):
    """
    Visualize the effect of data augmentation
    """
    train_transform, val_transform, _ = transforms_tuple
    
    # Sample some images
    sample_df = df.sample(min(num_samples, len(df)), random_state=SEED)
    
    fig, axes = plt.subplots(3, num_samples, figsize=(2*num_samples, 6))
    if num_samples == 1:
        axes = axes.reshape(3, 1)
    
    for i, (_, row) in enumerate(sample_df.iterrows()):
        if i >= num_samples:
            break
            
        # Load original image
        try:
            image = Image.open(row['image_path']).convert('RGB')
            
            # Original image
            axes[0, i].imshow(image)
            axes[0, i].set_title(f'Original\n{EMOTION_LABELS[row["expression"]]}', fontsize=9)
            axes[0, i].axis('off')
            
            # Validation transform (minimal processing)
            val_img = val_transform(image)
            val_img_display = val_img.permute(1, 2, 0)
            # Denormalize for display
            val_img_display = val_img_display * torch.tensor(IMAGENET_STD) + torch.tensor(IMAGENET_MEAN)
            val_img_display = torch.clamp(val_img_display, 0, 1)
            
            axes[1, i].imshow(val_img_display)
            axes[1, i].set_title('Val Transform', fontsize=9)
            axes[1, i].axis('off')
            
            # Training transform (with augmentation)
            train_img = train_transform(image)
            train_img_display = train_img.permute(1, 2, 0)
            # Denormalize for display
            train_img_display = train_img_display * torch.tensor(IMAGENET_STD) + torch.tensor(IMAGENET_MEAN)
            train_img_display = torch.clamp(train_img_display, 0, 1)
            
            axes[2, i].imshow(train_img_display)
            axes[2, i].set_title('Train Transform', fontsize=9)
            axes[2, i].axis('off')
            
        except Exception as e:
            print(f"Error processing image {row['image_path']}: {e}")
            continue
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'transform_examples.png', dpi=300, bbox_inches='tight')
    plt.show()

# Visualize transforms using VGG16 transforms (224x224)
print("\nVisualizing data augmentation effects:")
try:
    visualize_transforms(train_df_full, transforms_dict['vgg16'], num_samples=6)
except Exception as e:
    print(f"Could not visualize transforms: {e}")
    print("This might be due to missing image files - will continue anyway.")

print("Transforms setup completed!")

## 4. PyTorch Dataset & DataLoader

Custom dataset class for multi-task learning with facial expression recognition.

In [ ]:
class FERDataset(Dataset):
    """
    Custom Dataset for Facial Expression Recognition with Multi-Task Learning
    Returns: image, expression_label, valence_arousal_values
    """
    
    def __init__(self, df, transform=None, include_regression=True):
        """
        Args:
            df: DataFrame with columns ['image_path', 'expression', 'valence', 'arousal', 'valid_for_regression']
            transform: torchvision transforms to apply to images
            include_regression: whether to include regression targets (valence/arousal)
        """
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.include_regression = include_regression
        
        # Convert to numpy for faster access
        self.image_paths = df['image_path'].values
        self.expressions = df['expression'].values
        
        if include_regression:
            self.valences = df['valence'].values
            self.arousals = df['arousal'].values
            self.valid_for_regression = df['valid_for_regression'].values
        
        print(f"Dataset initialized with {len(self.df)} samples")
        if include_regression:
            valid_regression_count = sum(self.valid_for_regression)
            print(f"  - {valid_regression_count} samples valid for regression")
            print(f"  - {len(self.df) - valid_regression_count} samples for classification only")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Load image
        image_path = self.image_paths[idx]
        
        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            # Return a black image as fallback
            image = Image.new('RGB', (224, 224), color='black')
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        else:
            # Default: just convert to tensor
            image = transforms.ToTensor()(image)
        
        # Get expression label
        expression = torch.tensor(self.expressions[idx], dtype=torch.long)
        
        if self.include_regression:
            # Get valence and arousal
            valence = self.valences[idx]
            arousal = self.arousals[idx]
            valid_regression = self.valid_for_regression[idx]
            
            # Handle uncertain annotations (-2) or NaN values
            if not valid_regression or np.isnan(valence) or np.isnan(arousal):
                # Use neutral values for uncertain cases
                va_values = torch.tensor([0.0, 0.0], dtype=torch.float32)
                valid_regression = False
            else:
                va_values = torch.tensor([valence, arousal], dtype=torch.float32)
            
            return {
                'image': image,
                'expression': expression,
                'va_values': va_values,
                'valid_regression': torch.tensor(valid_regression, dtype=torch.bool),
                'image_path': image_path
            }
        else:
            return {
                'image': image,
                'expression': expression,
                'image_path': image_path
            }

def create_dataloaders(train_df, val_df, test_df, transforms_tuple, batch_size=32, num_workers=0):
    """
    Create DataLoaders for train, validation, and test sets
    Note: num_workers set to 0 to avoid pickle issues with transforms
    """
    train_transform, val_transform, test_transform = transforms_tuple
    
    # Create datasets
    train_dataset = FERDataset(train_df, transform=train_transform, include_regression=True)
    val_dataset = FERDataset(val_df, transform=val_transform, include_regression=True)
    test_dataset = FERDataset(test_df, transform=test_transform, include_regression=True)
    
    # Create dataloaders with num_workers=0 to avoid pickle issues
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=torch.cuda.is_available(),
        drop_last=True  # For consistent batch sizes during training
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=torch.cuda.is_available()
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=torch.cuda.is_available()
    )
    
    return train_loader, val_loader, test_loader, (train_dataset, val_dataset, test_dataset)

# Test the dataset with a small sample
print("Testing dataset implementation...")
try:
    # Create a small test dataset
    test_df_sample = train_df_full.head(5).copy()
    test_dataset = FERDataset(test_df_sample, transform=transforms_dict['vgg16'][0], include_regression=True)
    
    # Test loading a sample
    sample = test_dataset[0]
    print(f"Sample loaded successfully:")
    print(f"  Image shape: {sample['image'].shape}")
    print(f"  Expression: {sample['expression']} ({EMOTION_LABELS[sample['expression'].item()]})")
    print(f"  VA values: {sample['va_values']}")
    print(f"  Valid for regression: {sample['valid_regression']}")
    print(f"  Image path: {sample['image_path']}")
    
    print("Dataset implementation test passed!")
    
except Exception as e:
    print(f"Dataset test failed: {e}")
    print("This might be due to missing image files - will continue anyway.")

print("Dataset and DataLoader classes defined successfully!")

## 5. Metric Implementations

Comprehensive evaluation metrics for both classification and regression tasks.

In [ ]:
def concordance_correlation_coefficient(y_true, y_pred):
    """
    Calculate Lin's Concordance Correlation Coefficient (CCC)
    CCC = 2 * ρ * σ_x * σ_y / (σ_x² + σ_y² + (μ_x - μ_y)²)
    """
    # Convert to numpy if needed
    if torch.is_tensor(y_true):
        y_true = y_true.cpu().numpy()
    if torch.is_tensor(y_pred):
        y_pred = y_pred.cpu().numpy()
    
    # Remove NaN values
    mask = ~(np.isnan(y_true) | np.isnan(y_pred))
    if not mask.any():
        return np.nan
        
    y_true_clean = y_true[mask]
    y_pred_clean = y_pred[mask]
    
    if len(y_true_clean) < 2:
        return np.nan
    
    # Calculate means
    mean_true = np.mean(y_true_clean)
    mean_pred = np.mean(y_pred_clean)
    
    # Calculate variances
    var_true = np.var(y_true_clean)
    var_pred = np.var(y_pred_clean)
    
    # Calculate covariance
    covariance = np.mean((y_true_clean - mean_true) * (y_pred_clean - mean_pred))
    
    # Calculate CCC
    denominator = var_true + var_pred + (mean_true - mean_pred)**2
    if denominator == 0:
        return np.nan
        
    ccc = 2 * covariance / denominator
    return ccc

def sign_agreement_rate(y_true, y_pred, threshold=1e-6):
    """
    Calculate Sign Agreement Rate (SAGR)
    Measures agreement in the sign of predictions, ignoring values close to zero
    """
    if torch.is_tensor(y_true):
        y_true = y_true.cpu().numpy()
    if torch.is_tensor(y_pred):
        y_pred = y_pred.cpu().numpy()
    
    # Remove NaN values
    mask = ~(np.isnan(y_true) | np.isnan(y_pred))
    if not mask.any():
        return np.nan
        
    y_true_clean = y_true[mask]
    y_pred_clean = y_pred[mask]
    
    # Remove values close to zero (neutral zone)
    non_neutral_mask = (np.abs(y_true_clean) > threshold) & (np.abs(y_pred_clean) > threshold)
    
    if not non_neutral_mask.any():
        return np.nan
    
    y_true_filtered = y_true_clean[non_neutral_mask]
    y_pred_filtered = y_pred_clean[non_neutral_mask]
    
    # Calculate sign agreement
    sign_agreement = np.sign(y_true_filtered) == np.sign(y_pred_filtered)
    return np.mean(sign_agreement)

def krippendorff_alpha_nominal(y_true, y_pred):
    """
    Calculate Krippendorff's alpha for nominal data
    """
    try:
        if torch.is_tensor(y_true):
            y_true = y_true.cpu().numpy()
        if torch.is_tensor(y_pred):
            y_pred = y_pred.cpu().numpy()
        
        # Create reliability data matrix (2 coders x n observations)
        reliability_data = np.array([y_true, y_pred])
        
        # Calculate Krippendorff's alpha
        alpha = krippendorff.alpha(reliability_data, level_of_measurement='nominal')
        return alpha
    except Exception as e:
        print(f"Error calculating Krippendorff's alpha: {e}")
        return np.nan

class MetricsCalculator:
    """
    Comprehensive metrics calculator for multi-task learning
    """
    
    def __init__(self, num_classes=8):
        self.num_classes = num_classes
        
    def calculate_classification_metrics(self, y_true, y_pred, y_prob=None):
        """
        Calculate all classification metrics
        """
        if torch.is_tensor(y_true):
            y_true = y_true.cpu().numpy()
        if torch.is_tensor(y_pred):
            y_pred = y_pred.cpu().numpy()
        if y_prob is not None and torch.is_tensor(y_prob):
            y_prob = y_prob.cpu().numpy()
        
        metrics = {}
        
        try:
            # Basic metrics
            metrics['accuracy'] = accuracy_score(y_true, y_pred)
            metrics['f1_macro'] = f1_score(y_true, y_pred, average='macro', zero_division=0)
            metrics['f1_weighted'] = f1_score(y_true, y_pred, average='weighted', zero_division=0)
            
            # Cohen's Kappa
            metrics['cohen_kappa'] = cohen_kappa_score(y_true, y_pred)
            
            # Krippendorff's alpha (nominal)
            metrics['krippendorff_alpha'] = krippendorff_alpha_nominal(y_true, y_pred)
            
            if y_prob is not None:
                try:
                    # Multi-class ROC-AUC (One-vs-Rest)
                    lb = LabelBinarizer()
                    y_true_bin = lb.fit_transform(y_true)
                    if y_true_bin.shape[1] == 1:  # Binary case
                        y_true_bin = np.hstack([1 - y_true_bin, y_true_bin])
                    
                    metrics['roc_auc_ovr'] = roc_auc_score(y_true_bin, y_prob, average='macro', multi_class='ovr')
                    metrics['roc_auc_weighted'] = roc_auc_score(y_true_bin, y_prob, average='weighted', multi_class='ovr')
                    
                    # Average Precision (PR-AUC)
                    metrics['pr_auc_macro'] = average_precision_score(y_true_bin, y_prob, average='macro')
                    metrics['pr_auc_weighted'] = average_precision_score(y_true_bin, y_prob, average='weighted')
                    
                except Exception as e:
                    print(f"Error calculating AUC metrics: {e}")
                    metrics['roc_auc_ovr'] = np.nan
                    metrics['roc_auc_weighted'] = np.nan
                    metrics['pr_auc_macro'] = np.nan
                    metrics['pr_auc_weighted'] = np.nan
            
            # Per-class metrics
            class_report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
            for i in range(self.num_classes):
                if str(i) in class_report:
                    metrics[f'f1_class_{i}'] = class_report[str(i)]['f1-score']
                    metrics[f'precision_class_{i}'] = class_report[str(i)]['precision']
                    metrics[f'recall_class_{i}'] = class_report[str(i)]['recall']
            
        except Exception as e:
            print(f"Error calculating classification metrics: {e}")
            # Return default values
            for key in ['accuracy', 'f1_macro', 'f1_weighted', 'cohen_kappa', 'krippendorff_alpha']:
                metrics[key] = np.nan
        
        return metrics
    
    def calculate_regression_metrics(self, y_true, y_pred, mask=None):
        """
        Calculate all regression metrics
        """
        if torch.is_tensor(y_true):
            y_true = y_true.cpu().numpy()
        if torch.is_tensor(y_pred):
            y_pred = y_pred.cpu().numpy()
        if mask is not None and torch.is_tensor(mask):
            mask = mask.cpu().numpy()
        
        # Apply mask if provided (for filtering out uncertain annotations)
        if mask is not None:
            y_true = y_true[mask]
            y_pred = y_pred[mask]
        
        metrics = {}
        
        if len(y_true) == 0:
            return {k: np.nan for k in ['rmse', 'mae', 'pearson_r', 'pearson_p', 'sagr', 'ccc']}
        
        try:
            # RMSE and MAE
            metrics['rmse'] = np.sqrt(mean_squared_error(y_true, y_pred))
            metrics['mae'] = np.mean(np.abs(y_true - y_pred))
            
            # Pearson correlation
            if len(y_true) > 1 and np.var(y_true) > 0 and np.var(y_pred) > 0:
                pearson_r, pearson_p = pearsonr(y_true, y_pred)
                metrics['pearson_r'] = pearson_r
                metrics['pearson_p'] = pearson_p
            else:
                metrics['pearson_r'] = np.nan
                metrics['pearson_p'] = np.nan
            
            # Sign Agreement Rate (SAGR)
            metrics['sagr'] = sign_agreement_rate(y_true, y_pred)
            
            # Concordance Correlation Coefficient (CCC)
            metrics['ccc'] = concordance_correlation_coefficient(y_true, y_pred)
            
        except Exception as e:
            print(f"Error calculating regression metrics: {e}")
            metrics = {k: np.nan for k in ['rmse', 'mae', 'pearson_r', 'pearson_p', 'sagr', 'ccc']}
        
        return metrics
    
    def calculate_all_metrics(self, predictions, targets, valid_regression_mask=None):
        """
        Calculate all metrics for multi-task predictions
        
        Args:
            predictions: dict with 'classification' and 'regression' keys
            targets: dict with 'classification' and 'regression' keys
            valid_regression_mask: mask for valid regression samples
        """
        results = {}
        
        # Classification metrics
        if 'classification' in predictions and 'classification' in targets:
            cls_pred = predictions['classification']
            cls_target = targets['classification']
            cls_prob = predictions.get('classification_prob', None)
            
            results['classification'] = self.calculate_classification_metrics(
                cls_target, cls_pred, cls_prob
            )
        
        # Regression metrics (separate for valence and arousal)
        if 'regression' in predictions and 'regression' in targets:
            reg_pred = predictions['regression']  # Shape: (N, 2) for [valence, arousal]
            reg_target = targets['regression']    # Shape: (N, 2) for [valence, arousal]
            
            # Valence metrics
            results['valence'] = self.calculate_regression_metrics(
                reg_target[:, 0], reg_pred[:, 0], valid_regression_mask
            )
            
            # Arousal metrics
            results['arousal'] = self.calculate_regression_metrics(
                reg_target[:, 1], reg_pred[:, 1], valid_regression_mask
            )
            
            # Combined V-A metrics (mean of valence and arousal)
            va_ccc = (results['valence']['ccc'] + results['arousal']['ccc']) / 2
            results['combined_ccc'] = va_ccc
        
        return results

# Initialize metrics calculator
metrics_calculator = MetricsCalculator(num_classes=len(EMOTION_LABELS))

# Test metrics with dummy data
print("Testing metrics implementation...")
try:
    # Create dummy data
    n_samples = 100
    y_true_cls = np.random.randint(0, 8, n_samples)
    y_pred_cls = np.random.randint(0, 8, n_samples) 
    y_prob_cls = np.random.rand(n_samples, 8)
    y_prob_cls = y_prob_cls / y_prob_cls.sum(axis=1, keepdims=True)  # Normalize
    
    y_true_reg = np.random.uniform(-1, 1, (n_samples, 2))
    y_pred_reg = np.random.uniform(-1, 1, (n_samples, 2))
    valid_mask = np.random.choice([True, False], n_samples, p=[0.8, 0.2])
    
    # Test classification metrics
    cls_metrics = metrics_calculator.calculate_classification_metrics(
        y_true_cls, y_pred_cls, y_prob_cls
    )
    print("Classification metrics calculated successfully:")
    for k, v in list(cls_metrics.items())[:5]:  # Show first 5
        print(f"  {k}: {v:.4f}")
    
    # Test regression metrics
    val_metrics = metrics_calculator.calculate_regression_metrics(
        y_true_reg[:, 0], y_pred_reg[:, 0], valid_mask
    )
    print("\\nRegression metrics calculated successfully:")
    for k, v in val_metrics.items():
        print(f"  {k}: {v:.4f}")
    
    print("\\nMetrics implementation test passed!")
    
except Exception as e:
    print(f"Metrics test failed: {e}")

print("\\nMetrics calculator ready for use!")

## 6. Multi-Task Heads & Model Wrappers

Building multi-task learning models with shared backbones and separate heads for classification and regression.

In [ ]:
class MultiTaskHead(nn.Module):
    """
    Multi-task head with separate branches for classification and regression
    """
    def __init__(self, input_dim, num_classes=8, dropout_rate=0.3):
        super(MultiTaskHead, self).__init__()
        
        # Shared layers
        self.shared_layers = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate)
        )
        
        # Classification head (expression recognition)
        self.classification_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_classes)
        )
        
        # Regression head (valence/arousal prediction)
        self.regression_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 2),  # 2 outputs: valence and arousal
            nn.Tanh()  # Output between -1 and 1
        )
        
    def forward(self, x):
        # Shared feature extraction
        shared_features = self.shared_layers(x)
        
        # Task-specific predictions
        classification_logits = self.classification_head(shared_features)
        regression_outputs = self.regression_head(shared_features)
        
        return classification_logits, regression_outputs

class MultiTaskModel(nn.Module):
    """
    Multi-task learning model with various backbone architectures
    """
    def __init__(self, backbone_name='vgg16', num_classes=8, pretrained=True, dropout_rate=0.3):
        super(MultiTaskModel, self).__init__()
        
        self.backbone_name = backbone_name
        self.num_classes = num_classes
        
        # Initialize backbone
        if backbone_name == 'vgg16':
            self.backbone = models.vgg16_bn(pretrained=pretrained)
            feature_dim = self.backbone.classifier[0].in_features
            # Remove the classifier
            self.backbone.classifier = nn.Identity()
            
        elif backbone_name == 'efficientnet_b0':
            self.backbone = timm.create_model('efficientnet_b0', pretrained=pretrained)
            feature_dim = self.backbone.classifier.in_features
            # Remove the classifier
            self.backbone.classifier = nn.Identity()
            
        elif backbone_name == 'efficientnet_b2':
            self.backbone = timm.create_model('efficientnet_b2', pretrained=pretrained)
            feature_dim = self.backbone.classifier.in_features
            # Remove the classifier
            self.backbone.classifier = nn.Identity()
            
        elif backbone_name == 'convnext_tiny':
            self.backbone = timm.create_model('convnext_tiny', pretrained=pretrained)
            feature_dim = self.backbone.head.in_features
            # Remove the head
            self.backbone.head = nn.Identity()
            
        else:
            raise ValueError(f"Unsupported backbone: {backbone_name}")
        
        # Multi-task head
        self.head = MultiTaskHead(feature_dim, num_classes, dropout_rate)
        
        # Store feature dimension for reference
        self.feature_dim = feature_dim
        
        print(f"Initialized {backbone_name} with feature dimension: {feature_dim}")
    
    def forward(self, x):
        # Extract features using backbone
        features = self.backbone(x)
        
        # Flatten if necessary (for VGG16)
        if len(features.shape) > 2:
            features = torch.flatten(features, 1)
        
        # Multi-task predictions
        classification_logits, regression_outputs = self.head(features)
        
        return classification_logits, regression_outputs
    
    def freeze_backbone(self):
        """Freeze backbone parameters"""
        for param in self.backbone.parameters():
            param.requires_grad = False
        print(f"Frozen {self.backbone_name} backbone")
    
    def unfreeze_backbone(self):
        """Unfreeze backbone parameters"""
        for param in self.backbone.parameters():
            param.requires_grad = True
        print(f"Unfrozen {self.backbone_name} backbone")
    
    def freeze_early_layers(self, freeze_ratio=0.5):
        """Freeze early layers of the backbone"""
        total_params = sum(1 for _ in self.backbone.parameters())
        freeze_count = int(total_params * freeze_ratio)
        
        for i, param in enumerate(self.backbone.parameters()):
            if i < freeze_count:
                param.requires_grad = False
            else:
                param.requires_grad = True
        
        print(f"Frozen first {freeze_count}/{total_params} layers of {self.backbone_name}")
    
    def get_parameter_count(self):
        """Get the number of trainable parameters"""
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        
        return {
            'total_parameters': total_params,
            'trainable_parameters': trainable_params,
            'frozen_parameters': total_params - trainable_params
        }

def create_model(backbone_name, num_classes=8, pretrained=True, dropout_rate=0.3):
    """
    Factory function to create multi-task models
    """
    model = MultiTaskModel(
        backbone_name=backbone_name,
        num_classes=num_classes,
        pretrained=pretrained,
        dropout_rate=dropout_rate
    )
    return model

# Test model creation
print("Testing model creation...")
test_models = {}

for backbone_name in ['vgg16', 'efficientnet_b0', 'efficientnet_b2', 'convnext_tiny']:
    try:
        print(f"\\nCreating {backbone_name} model...")
        model = create_model(backbone_name, num_classes=len(EMOTION_LABELS))
        
        # Test forward pass
        dummy_input = torch.randn(2, 3, 224, 224)
        if backbone_name == 'efficientnet_b2':
            dummy_input = torch.randn(2, 3, 260, 260)  # EfficientNet-B2 uses 260x260
        
        with torch.no_grad():
            cls_logits, reg_outputs = model(dummy_input)
        
        print(f"  Classification output shape: {cls_logits.shape}")
        print(f"  Regression output shape: {reg_outputs.shape}")
        
        # Get parameter count
        param_info = model.get_parameter_count()
        print(f"  Total parameters: {param_info['total_parameters']:,}")
        print(f"  Trainable parameters: {param_info['trainable_parameters']:,}")
        
        test_models[backbone_name] = model
        
    except Exception as e:
        print(f"  Error creating {backbone_name}: {e}")

print(f"\\nSuccessfully created {len(test_models)} models!")
print("Available models:", list(test_models.keys()))

In [ ]:
# Unified Accuracy-First Phased Trainer (replaces earlier trainer classes)
from dataclasses import dataclass
from typing import Dict, List, Optional

@dataclass
class PhaseConfig:
    name: str
    epochs: int
    freeze_backbone: bool
    backbone_lr: float
    head_lr: float
    lambda_reg_schedule: Dict[int, float]
    weight_decay: float
    label_smoothing: float = 0.05
    mixup_alpha: Optional[float] = None
    cutmix_alpha: Optional[float] = None
    grad_clip: float = 1.0
    cosine_T_max: Optional[int] = None
    cosine_min_lr: float = 1e-6

PHASES: List[PhaseConfig] = [
    PhaseConfig(
        name="warmup_cls_only",
        epochs=6,
        freeze_backbone=True,
        backbone_lr=0.0,
        head_lr=1e-3,
        lambda_reg_schedule={0: 0.0},
        weight_decay=0.01,
        label_smoothing=0.05,
        mixup_alpha=0.2,
    ),
    PhaseConfig(
        name="multitask_finetune",
        epochs=24,
        freeze_backbone=False,
        backbone_lr=5e-5,
        head_lr=2.5e-4,
        lambda_reg_schedule={0:0.0, 2:0.01, 4:0.03, 6:0.05},
        weight_decay=0.005,
        label_smoothing=0.03,
        mixup_alpha=0.1,
        cutmix_alpha=0.1,
        cosine_T_max=24,
        cosine_min_lr=1e-6,
    ),
]

def get_lambda_reg(phase: PhaseConfig, local_epoch: int) -> float:
    keys = sorted(phase.lambda_reg_schedule.keys())
    current = phase.lambda_reg_schedule[keys[0]]
    for k in keys:
        if local_epoch >= k:
            current = phase.lambda_reg_schedule[k]
        else:
            break
    return current

class SmoothedCELoss(nn.Module):
    def __init__(self, smoothing=0.05):
        super().__init__()
        self.smoothing = smoothing
    def forward(self, logits, target):
        if self.smoothing <= 0:
            return F.cross_entropy(logits, target)
        n_classes = logits.size(-1)
        log_probs = F.log_softmax(logits, dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(log_probs)
            true_dist.fill_(self.smoothing / (n_classes - 1))
            true_dist.scatter_(1, target.unsqueeze(1), 1 - self.smoothing)
        return (-true_dist * log_probs).sum(dim=-1).mean()

def apply_mixup_cutmix(images, targets, num_classes, mixup_alpha=None, cutmix_alpha=None):
    if mixup_alpha is None and cutmix_alpha is None:
        return images, targets, None
    lam = 1.0
    mixed_targets = None
    batch_size = images.size(0)
    index = torch.randperm(batch_size).to(images.device)
    if cutmix_alpha and torch.rand(1).item() < 0.5:
        lam = np.random.beta(cutmix_alpha, cutmix_alpha)
        W, H = images.size(3), images.size(2)
        cut_rat = np.sqrt(1. - lam)
        cut_w = int(W * cut_rat)
        cut_h = int(H * cut_rat)
        cx = np.random.randint(W)
        cy = np.random.randint(H)
        x1 = np.clip(cx - cut_w // 2, 0, W)
        x2 = np.clip(cx + cut_w // 2, 0, W)
        y1 = np.clip(cy - cut_h // 2, 0, H)
        y2 = np.clip(cy + cut_h // 2, 0, H)
        images[:, :, y1:y2, x1:x2] = images[index, :, y1:y2, x1:x2]
        lam = 1 - ((x2 - x1) * (y2 - y1) / (W * H + 1e-6))
    elif mixup_alpha:
        lam = np.random.beta(mixup_alpha, mixup_alpha)
        images = lam * images + (1 - lam) * images[index, :]
    mixed_targets = (targets, targets[index], lam)
    return images, targets, mixed_targets

class AccuracyFirstTrainer:
    def __init__(self, model, train_loader, val_loader, device, num_classes=8):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.num_classes = num_classes
        self.best_state = None
        self.best_acc = 0.0
        self.history = []

    def _build_optimizer(self, phase: PhaseConfig):
        head_params = list(self.model.head.parameters()) if hasattr(self.model, 'head') else []
        backbone_params = [p for n,p in self.model.named_parameters() if 'head' not in n]
        param_groups = []
        if phase.backbone_lr > 0 and not phase.freeze_backbone:
            param_groups.append({'params': backbone_params, 'lr': phase.backbone_lr})
        else:
            for p in backbone_params:
                p.requires_grad = not phase.freeze_backbone
        if head_params:
            param_groups.append({'params': head_params, 'lr': phase.head_lr})
        optimizer = torch.optim.AdamW(param_groups, weight_decay=phase.weight_decay, betas=(0.9,0.999))
        if phase.cosine_T_max:
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer, T_max=phase.cosine_T_max, eta_min=phase.cosine_min_lr
            )
        else:
            scheduler = None
        return optimizer, scheduler

    def _classification_loss(self, logits, targets, phase: PhaseConfig, mix_targets=None):
        criterion = SmoothedCELoss(phase.label_smoothing)
        if mix_targets is None:
            return criterion(logits, targets)
        t1, t2, lam = mix_targets
        return lam * criterion(logits, t1) + (1 - lam) * criterion(logits, t2)

    def _regression_loss(self, preds, targets, valid_mask):
        if valid_mask.sum() == 0:
            return preds.new_tensor(0.0)
        diff = preds[valid_mask] - targets[valid_mask]
        return F.smooth_l1_loss(diff, torch.zeros_like(diff))

    def train_phase(self, phase: PhaseConfig, global_epoch_start: int):
        print(f"\n=== Phase: {phase.name} ({phase.epochs} epochs) ===")
        optimizer, scheduler = self._build_optimizer(phase)
        for local_epoch in range(phase.epochs):
            lambda_reg = get_lambda_reg(phase, local_epoch)
            self.model.train()
            train_cls_loss = train_reg_loss = 0.0
            correct = total = 0
            for batch in self.train_loader:
                images = batch['image'].to(self.device)
                labels = batch['expression'].to(self.device)
                va = batch['va_values'].to(self.device)
                valid_reg = batch['valid_regression'].to(self.device)

                images, labels, mix_targets = apply_mixup_cutmix(
                    images, labels, self.num_classes,
                    mixup_alpha=phase.mixup_alpha, cutmix_alpha=phase.cutmix_alpha
                )

                optimizer.zero_grad(set_to_none=True)
                cls_logits, reg_out = self.model(images)
                cls_loss = self._classification_loss(cls_logits, labels, phase, mix_targets)
                reg_loss = self._regression_loss(reg_out, va, valid_reg) if lambda_reg > 0 else cls_logits.new_tensor(0.0)
                loss = cls_loss + lambda_reg * reg_loss
                loss.backward()
                if phase.grad_clip:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), phase.grad_clip)
                optimizer.step()
                train_cls_loss += cls_loss.item()
                train_reg_loss += reg_loss.item()
                preds = cls_logits.argmax(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
            if scheduler: scheduler.step()
            train_acc = correct / max(1,total)
            val_acc, val_f1 = self.validate()
            print(f"Epoch {global_epoch_start + local_epoch+1} | TrainAcc {train_acc:.3f} | ValAcc {val_acc:.3f} | ValF1 {val_f1:.3f} | λ={lambda_reg:.3f}")
            self.history.append({
                'phase': phase.name,
                'epoch': global_epoch_start + local_epoch + 1,
                'train_acc': train_acc,
                'val_acc': val_acc,
                'val_f1': val_f1,
                'lambda_reg': lambda_reg
            })
            if val_acc > self.best_acc:
                self.best_acc = val_acc
                self.best_state = {k:v.cpu().clone() for k,v in self.model.state_dict().items()}
                print(f"  ✅ New best accuracy: {val_acc:.3f}")
        return global_epoch_start + phase.epochs

    def validate(self):
        self.model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in self.val_loader:
                images = batch['image'].to(self.device)
                labels = batch['expression'].to(self.device)
                cls_logits, _ = self.model(images)
                preds = cls_logits.argmax(1)
                all_preds.append(preds.cpu())
                all_labels.append(labels.cpu())
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)
        acc = (all_preds == all_labels).float().mean().item()
        try:
            f1 = f1_score(all_labels.numpy(), all_preds.numpy(), average='weighted')
        except Exception:
            f1 = 0.0
        return acc, f1

    def run(self):
        global_epoch = 0
        for phase in PHASES:
            global_epoch = self.train_phase(phase, global_epoch)
        if self.best_state:
            self.model.load_state_dict(self.best_state)
        print(f"\nBest validation accuracy achieved: {self.best_acc:.3f}")
        return self.history

print("Unified AccuracyFirstTrainer ready. Usage example:\n"
      "model = create_model('vgg16', num_classes=len(EMOTION_LABELS), pretrained=True)\n"
      "train_loader, val_loader, _, _ = create_dataloaders(train_df_full, val_df_full, test_df_full, transforms_dict['vgg16'], batch_size=16)\n"
      "trainer = AccuracyFirstTrainer(model, train_loader, val_loader, device)\n"
      "trainer.run()")

## 7. Training Strategy (Accuracy-First Phased Trainer)

To simplify and clean the workflow we adopt a single unified trainer that:

1. Runs an explicit classification-only warmup (freezes backbone, λ_reg = 0) to rapidly specialize the new multi-task head.
2. Progressively unfreezes and fine-tunes the backbone while ramping the regression (valence/arousal) loss weight.
3. Supports light Mixup / CutMix (configurable per phase) and label smoothing.
4. Tracks best validation accuracy and restores those weights at the end.

This replaces previous ad‑hoc trainer variants (ImprovedTrainer, legacy joint trainer) to keep the notebook coherent.


In [ ]:
# Multi-Task Loss Function
class MultiTaskLoss(nn.Module):
    """Combined loss for classification and regression tasks"""
    
    def __init__(self, lambda_reg=1.0, class_weights=None):
        super().__init__()
        self.lambda_reg = lambda_reg
        self.class_weights = class_weights
        
        # Classification loss
        if class_weights is not None:
            self.cls_criterion = nn.CrossEntropyLoss(weight=class_weights)
        else:
            self.cls_criterion = nn.CrossEntropyLoss()
        
        # Regression loss (smooth L1 loss is more robust than MSE)
        self.reg_criterion = nn.SmoothL1Loss()
    
    def forward(self, cls_logits, reg_outputs, cls_targets, reg_targets, valid_regression):
        """
        Args:
            cls_logits: Classification predictions [batch_size, num_classes]
            reg_outputs: Regression predictions [batch_size, 2] (valence, arousal)
            cls_targets: Classification ground truth [batch_size]
            reg_targets: Regression ground truth [batch_size, 2] (valence, arousal)
            valid_regression: Mask for valid regression samples [batch_size]
        """
        # Classification loss (all samples)
        cls_loss = self.cls_criterion(cls_logits, cls_targets)
        
        # Regression loss (only valid samples)
        if valid_regression.sum() > 0:
            valid_reg_pred = reg_outputs[valid_regression]
            valid_reg_targets = reg_targets[valid_regression]
            reg_loss = self.reg_criterion(valid_reg_pred, valid_reg_targets)
        else:
            reg_loss = torch.tensor(0.0, device=cls_logits.device)
        
        # Combined loss
        total_loss = cls_loss + self.lambda_reg * reg_loss
        
        return total_loss, cls_loss, reg_loss

# Fixed Multi-Task Model Factory
def create_model(model_name, num_classes=8, pretrained=True, dropout_rate=0.3):
    """
    Create multi-task model with proper feature dimension handling
    """
    model_name = model_name.lower()
    
    if model_name == 'vgg16':
        backbone = models.vgg16_bn(weights='IMAGENET1K_V1' if pretrained else None)
        # VGG16 classifier: features -> avgpool -> classifier
        # Feature dim from classifier[0] (first linear layer)
        feature_dim = backbone.classifier[0].in_features  # Should be 25088
        backbone.classifier = nn.Identity()  # Remove classifier
        
    elif model_name == 'efficientnet_b0':
        backbone = timm.create_model('efficientnet_b0', pretrained=pretrained)
        feature_dim = backbone.classifier.in_features
        backbone.classifier = nn.Identity()
        
    elif model_name == 'efficientnet_b2':
        backbone = timm.create_model('efficientnet_b2', pretrained=pretrained)  
        feature_dim = backbone.classifier.in_features
        backbone.classifier = nn.Identity()
        
    elif model_name == 'convnext_tiny':
        backbone = timm.create_model('convnext_tiny', pretrained=pretrained)
        feature_dim = backbone.head.fc.in_features
        backbone.head = nn.Identity()
        
    else:
        raise ValueError(f"Unsupported model: {model_name}")
    
    # Create multi-task head
    head = MultiTaskHead(feature_dim, num_classes, dropout_rate)
    
    # Wrap in MultiTaskModel
    model = MultiTaskModel(backbone, head)
    
    print(f"Created {model_name} with feature_dim={feature_dim}")
    return model

class Trainer:
    """Standard trainer for multi-task learning"""
    
    def __init__(self, model, train_loader, val_loader, device, config=None):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        
        # Default config
        default_config = {
            'epochs': 30,
            'lr': 3e-4,
            'weight_decay': 1e-4,
            'lambda_reg': 1.0,
            'patience': 15,
            'warmup_epochs': 2,
        }
        self.config = {**default_config, **(config or {})}
        
        # Initialize criterion and optimizer
        self.criterion = MultiTaskLoss(lambda_reg=self.config['lambda_reg'])
        self.optimizer = optim.AdamW(
            self.model.parameters(),
            lr=self.config['lr'],
            weight_decay=self.config['weight_decay']
        )
        
        # Learning rate scheduler
        self.scheduler = optim.lr_scheduler.OneCycleLR(
            self.optimizer,
            max_lr=self.config['lr'],
            epochs=self.config['epochs'],
            steps_per_epoch=len(self.train_loader),
            pct_start=0.3,
            anneal_strategy='cos'
        )
        
        # Early stopping
        self.best_ccc = -float('inf')
        self.patience_counter = 0
        self.best_model_state = None
        
        # Training history
        self.history = {
            'train_loss': [], 'val_loss': [],
            'val_accuracy': [], 'val_f1': [], 'val_ccc': []
        }
    
    def train_epoch(self):
        """Train for one epoch"""
        self.model.train()
        total_loss = 0.0
        num_batches = 0
        
        for batch in tqdm(self.train_loader, desc='Training'):
            images = batch['image'].to(self.device)
            expressions = batch['expression'].to(self.device)
            va_values = batch['va_values'].to(self.device)
            valid_regression = batch['valid_regression'].to(self.device)
            
            self.optimizer.zero_grad()
            
            # Forward pass
            cls_logits, reg_outputs = self.model(images)
            
            # Calculate loss
            loss, cls_loss, reg_loss = self.criterion(
                cls_logits, reg_outputs, expressions, va_values, valid_regression
            )
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            self.scheduler.step()
            
            total_loss += loss.item()
            num_batches += 1
        
        return total_loss / num_batches
    
    def validate_epoch(self):
        """Validate for one epoch"""
        self.model.eval()
        total_loss = 0.0
        all_preds = []
        all_labels = []
        all_va_preds = []
        all_va_true = []
        all_valid_regression = []
        
        with torch.no_grad():
            for batch in tqdm(self.val_loader, desc='Validation'):
                images = batch['image'].to(self.device)
                expressions = batch['expression'].to(self.device)
                va_values = batch['va_values'].to(self.device)
                valid_regression = batch['valid_regression'].to(self.device)
                
                cls_logits, reg_outputs = self.model(images)
                loss, _, _ = self.criterion(
                    cls_logits, reg_outputs, expressions, va_values, valid_regression
                )
                
                total_loss += loss.item()
                
                # Collect predictions
                preds = torch.argmax(cls_logits, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(expressions.cpu().numpy())
                all_va_preds.extend(reg_outputs.cpu().numpy())
                all_va_true.extend(va_values.cpu().numpy())
                all_valid_regression.extend(valid_regression.cpu().numpy())
        
        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')
        
        # Calculate CCC for regression (only valid samples)
        all_va_preds = np.array(all_va_preds)
        all_va_true = np.array(all_va_true)
        all_valid_regression = np.array(all_valid_regression)
        
        if np.any(all_valid_regression):
            valid_mask = all_valid_regression
            val_preds = all_va_preds[valid_mask, 0]  # valence
            val_true = all_va_true[valid_mask, 0]
            aro_preds = all_va_preds[valid_mask, 1]  # arousal
            aro_true = all_va_true[valid_mask, 1]
            
            val_ccc = concordance_correlation_coefficient(val_true, val_preds)
            aro_ccc = concordance_correlation_coefficient(aro_true, aro_preds)
            combined_ccc = (val_ccc + aro_ccc) / 2
        else:
            val_ccc = aro_ccc = combined_ccc = 0.0
        
        return {
            'val_loss': total_loss / len(self.val_loader),
            'accuracy': accuracy,
            'f1': f1,
            'val_ccc': val_ccc,
            'aro_ccc': aro_ccc,
            'combined_ccc': combined_ccc
        }
    
    def train(self):
        """Full training loop"""
        print(f"Starting training for {self.config['epochs']} epochs...")
        
        # Warmup: freeze backbone for first few epochs
        if self.config['warmup_epochs'] > 0:
            self.model.freeze_backbone()
            print(f"Warmup: freezing backbone for {self.config['warmup_epochs']} epochs")
        
        for epoch in range(1, self.config['epochs'] + 1):
            print(f"\nEpoch {epoch}/{self.config['epochs']}")
            
            # Unfreeze after warmup
            if epoch == self.config['warmup_epochs'] + 1:
                self.model.unfreeze_backbone()
                print("Unfrozing backbone - training all parameters")
            
            # Train and validate
            train_loss = self.train_epoch()
            val_metrics = self.validate_epoch()
            
            # Print metrics
            print(f"Train Loss: {train_loss:.4f}")
            print(f"Val Loss: {val_metrics['val_loss']:.4f}")
            print(f"Val Accuracy: {val_metrics['accuracy']:.4f}")
            print(f"Val F1: {val_metrics['f1']:.4f}")
            print(f"Val CCC - Valence: {val_metrics['val_ccc']:.4f}")
            print(f"Val CCC - Arousal: {val_metrics['aro_ccc']:.4f}")
            print(f"Combined CCC: {val_metrics['combined_ccc']:.4f}")
            
            # Save history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_metrics['val_loss'])
            self.history['val_accuracy'].append(val_metrics['accuracy'])
            self.history['val_f1'].append(val_metrics['f1'])
            self.history['val_ccc'].append(val_metrics['combined_ccc'])
            
            # Early stopping and model saving
            current_ccc = val_metrics['combined_ccc']
            if current_ccc > self.best_ccc:
                self.best_ccc = current_ccc
                self.best_model_state = self.model.state_dict().copy()
                self.patience_counter = 0
                print(f"💾 Best model saved (CCC: {current_ccc:.4f})")
            else:
                self.patience_counter += 1
                if self.patience_counter >= self.config['patience']:
                    print(f"Early stopping triggered after {epoch} epochs")
                    break
        
        # Load best model
        if self.best_model_state is not None:
            self.model.load_state_dict(self.best_model_state)
            print(f"Loaded best model with CCC: {self.best_ccc:.4f}")
        
        return self.history

print("✅ Fixed MultiTaskLoss, create_model factory, and standard Trainer classes ready!")

In [ ]:
# Consolidated PhaseConfig + AccuracyFirstTrainer
from dataclasses import dataclass
from typing import List, Optional, Tuple
import math
import torch
import torch.nn.functional as F

@dataclass
class PhaseConfig:
    name: str
    epochs: int
    freeze_backbone: bool
    lr_head: float
    lr_backbone: float
    lambda_reg_max: float
    lambda_reg_start: float = 0.0
    mixup_alpha: float = 0.0
    cutmix_alpha: float = 0.0
    label_smoothing: float = 0.0
    cosine: bool = True

PHASES: List[PhaseConfig] = [
    PhaseConfig(
        name="warmup",
        epochs=2,
        freeze_backbone=True,
        lr_head=5e-4,
        lr_backbone=1e-5,
        lambda_reg_max=0.0,
        lambda_reg_start=0.0,
        label_smoothing=0.1,
    ),
    PhaseConfig(
        name="head_focus",
        epochs=4,
        freeze_backbone=False,
        lr_head=4e-4,
        lr_backbone=2e-5,
        lambda_reg_max=0.05,
        lambda_reg_start=0.0,
        label_smoothing=0.1,
        mixup_alpha=0.2,
        cutmix_alpha=0.0,
    ),
    PhaseConfig(
        name="joint_finetune",
        epochs=6,
        freeze_backbone=False,
        lr_head=3e-4,
        lr_backbone=1e-5,
        lambda_reg_max=0.1,
        lambda_reg_start=0.05,
        label_smoothing=0.05,
        mixup_alpha=0.0,
        cutmix_alpha=0.0,
    ),
]

class AccuracyFirstTrainer:
    """Phased trainer focusing on classification accuracy first"""
    
    def __init__(self, model, train_loader, val_loader, device):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.history = []
        self.best_acc = 0.0
        
    def train(self):
        """Run all training phases"""
        global_epoch = 0
        
        for phase in PHASES:
            print(f"\n{'='*50}")
            print(f"Phase: {phase.name} ({phase.epochs} epochs)")
            print(f"{'='*50}")
            
            # Setup optimizer for this phase
            if phase.freeze_backbone:
                # Freeze backbone, only train head
                for param in self.model.backbone.parameters():
                    param.requires_grad = False
                for param in self.model.head.parameters():
                    param.requires_grad = True
                    
                optimizer = torch.optim.AdamW(
                    self.model.head.parameters(),
                    lr=phase.lr_head,
                    weight_decay=0.01
                )
            else:
                # Train both backbone and head with different LRs
                for param in self.model.parameters():
                    param.requires_grad = True
                    
                optimizer = torch.optim.AdamW([
                    {'params': self.model.backbone.parameters(), 'lr': phase.lr_backbone},
                    {'params': self.model.head.parameters(), 'lr': phase.lr_head}
                ], weight_decay=0.01)
            
            # Setup scheduler
            if phase.cosine:
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                    optimizer, T_max=phase.epochs, eta_min=1e-7
                )
            else:
                scheduler = None
            
            # Train for this phase
            for epoch in range(phase.epochs):
                global_epoch += 1
                
                # Calculate lambda_reg for this epoch
                if phase.epochs > 1:
                    progress = epoch / (phase.epochs - 1)
                else:
                    progress = 1.0
                lambda_reg = phase.lambda_reg_start + progress * (phase.lambda_reg_max - phase.lambda_reg_start)
                
                # Train one epoch
                train_loss, train_acc = self._train_epoch(optimizer, phase, lambda_reg)
                
                # Validate
                val_loss, val_acc, val_f1 = self._validate_epoch(lambda_reg)
                
                # Update scheduler
                if scheduler:
                    scheduler.step()
                
                # Save best model
                if val_acc > self.best_acc:
                    self.best_acc = val_acc
                    print(f"  ✅ New best accuracy: {val_acc:.4f}")
                
                # Log results
                print(f"Epoch {global_epoch:2d} | Loss: {train_loss:.4f} | "
                      f"TrainAcc: {train_acc:.4f} | ValAcc: {val_acc:.4f} | "
                      f"ValF1: {val_f1:.4f} | λ: {lambda_reg:.3f}")
                
                self.history.append({
                    'epoch': global_epoch,
                    'phase': phase.name,
                    'train_loss': train_loss,
                    'train_acc': train_acc,
                    'val_loss': val_loss,
                    'val_acc': val_acc,
                    'val_f1': val_f1,
                    'lambda_reg': lambda_reg
                })
        
        print(f"\n🎉 Training completed! Best accuracy: {self.best_acc:.4f}")
        return self.history
    
    def _train_epoch(self, optimizer, phase, lambda_reg):
        self.model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        
        for batch in self.train_loader:
            images = batch['image'].to(self.device)
            labels = batch['expression'].to(self.device)
            va_values = batch['va_values'].to(self.device)
            valid_reg = batch['valid_regression'].to(self.device)
            
            optimizer.zero_grad()
            
            # Forward pass
            cls_logits, reg_outputs = self.model(images)
            
            # Classification loss
            if phase.label_smoothing > 0:
                cls_loss = self._label_smooth_loss(cls_logits, labels, phase.label_smoothing)
            else:
                cls_loss = F.cross_entropy(cls_logits, labels)
            
            # Regression loss
            if lambda_reg > 0 and valid_reg.sum() > 0:
                reg_pred = reg_outputs[valid_reg]
                reg_true = va_values[valid_reg]
                reg_loss = F.smooth_l1_loss(reg_pred, reg_true)
            else:
                reg_loss = torch.tensor(0.0, device=self.device)
            
            # Combined loss
            loss = cls_loss + lambda_reg * reg_loss
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            optimizer.step()
            
            # Statistics
            total_loss += loss.item()
            _, predicted = cls_logits.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        return total_loss / len(self.train_loader), correct / total
    
    def _validate_epoch(self, lambda_reg):
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for batch in self.val_loader:
                images = batch['image'].to(self.device)
                labels = batch['expression'].to(self.device)
                va_values = batch['va_values'].to(self.device)
                valid_reg = batch['valid_regression'].to(self.device)
                
                cls_logits, reg_outputs = self.model(images)
                
                # Classification loss
                cls_loss = F.cross_entropy(cls_logits, labels)
                
                # Regression loss
                if lambda_reg > 0 and valid_reg.sum() > 0:
                    reg_pred = reg_outputs[valid_reg]
                    reg_true = va_values[valid_reg]
                    reg_loss = F.smooth_l1_loss(reg_pred, reg_true)
                else:
                    reg_loss = torch.tensor(0.0, device=self.device)
                
                loss = cls_loss + lambda_reg * reg_loss
                
                total_loss += loss.item()
                _, predicted = cls_logits.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        accuracy = correct / total
        f1 = f1_score(all_labels, all_preds, average='weighted')
        
        return total_loss / len(self.val_loader), accuracy, f1
    
    def _label_smooth_loss(self, logits, targets, smoothing=0.1):
        """Label smoothing cross entropy loss"""
        log_probs = F.log_softmax(logits, dim=-1)
        num_classes = logits.size(-1)
        
        with torch.no_grad():
            true_dist = torch.zeros_like(log_probs)
            true_dist.fill_(smoothing / (num_classes - 1))
            true_dist.scatter_(1, targets.unsqueeze(1), 1.0 - smoothing)
        
        return (-true_dist * log_probs).sum(dim=-1).mean()

print("✅ AccuracyFirstTrainer class ready!")
print("Usage: trainer = AccuracyFirstTrainer(model, train_loader, val_loader, device)")
print("       history = trainer.train()")

## 7. Losses & Optimizers

Setting up multi-task loss functions, optimizers, and training utilities.

## Accuracy-First Phased Trainer (Consolidated)
This section introduces a structured phased training strategy prioritized for classification convergence before gradually integrating the regression objectives. It replaces earlier scattered draft cells.

Phases:
1. Warmup (classification only, backbone frozen)
2. Head Focus (classification dominant, light regression ramp)
3. Joint Fine-tune (balanced multi-task)

Key features: differential learning rates, lambda_reg ramp, optional mixup/cutmix, cosine schedule per phase.


In [ ]:
class MultiTaskLoss(nn.Module):
    """
    Combined loss for multi-task learning
    """
    def __init__(self, lambda_reg=1.0, class_weights=None):
        super(MultiTaskLoss, self).__init__()
        self.lambda_reg = lambda_reg
        
        # Classification loss
        if class_weights is not None:
            self.cls_criterion = nn.CrossEntropyLoss(weight=class_weights)
        else:
            self.cls_criterion = nn.CrossEntropyLoss()
        
        # Regression loss (smooth L1 is less sensitive to outliers than MSE)
        self.reg_criterion = nn.SmoothL1Loss(reduction='none')
        
    def forward(self, cls_logits, reg_outputs, cls_targets, reg_targets, valid_regression):
        """
        Calculate combined loss
        
        Args:
            cls_logits: Classification predictions (N, num_classes)
            reg_outputs: Regression predictions (N, 2) for [valence, arousal]
            cls_targets: Classification targets (N,)
            reg_targets: Regression targets (N, 2)
            valid_regression: Binary mask for valid regression samples (N,)
        """
        # Classification loss (all samples)
        cls_loss = self.cls_criterion(cls_logits, cls_targets)
        
        # Regression loss (only valid samples)
        reg_loss = torch.tensor(0.0, device=cls_logits.device)
        if valid_regression.any():
            valid_mask = valid_regression.bool()
            reg_loss_per_sample = self.reg_criterion(
                reg_outputs[valid_mask], 
                reg_targets[valid_mask]
            )
            # Average over dimensions and samples
            reg_loss = reg_loss_per_sample.mean()
        
        # Combined loss
        total_loss = cls_loss + self.lambda_reg * reg_loss
        
        return {
            'total_loss': total_loss,
            'classification_loss': cls_loss,
            'regression_loss': reg_loss
        }

class EarlyStopping:
    """Early stopping utility"""
    def __init__(self, patience=10, min_delta=1e-4, mode='max'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        
    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif self.mode == 'max':
            if score > self.best_score + self.min_delta:
                self.best_score = score
                self.counter = 0
            else:
                self.counter += 1
        else:  # mode == 'min'
            if score < self.best_score - self.min_delta:
                self.best_score = score
                self.counter = 0
            else:
                self.counter += 1
        
        if self.counter >= self.patience:
            self.early_stop = True
        
        return self.early_stop

class Trainer:
    """
    Multi-task model trainer
    """
    def __init__(self, model, train_loader, val_loader, device, 
                 lambda_reg=1.0, lr=3e-4, weight_decay=1e-4,
                 patience=15, class_weights=None):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        
        # Loss function
        if class_weights is not None:
            class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
        self.criterion = MultiTaskLoss(lambda_reg=lambda_reg, class_weights=class_weights)
        
        # Optimizer
        self.optimizer = torch.optim.AdamW(
            model.parameters(), 
            lr=lr, 
            weight_decay=weight_decay
        )
        
        # Learning rate scheduler
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(
            self.optimizer,
            max_lr=lr,
            epochs=50,  # Will be updated in train method
            steps_per_epoch=len(train_loader),
            pct_start=0.1,
            anneal_strategy='cos'
        )
        
        # Early stopping
        self.early_stopping = EarlyStopping(patience=patience, mode='max')
        
        # Metrics calculator
        self.metrics_calculator = MetricsCalculator(num_classes=8)
        
        # Training history
        self.history = {
            'train_loss': [], 'val_loss': [],
            'train_cls_loss': [], 'val_cls_loss': [],
            'train_reg_loss': [], 'val_reg_loss': [],
            'val_accuracy': [], 'val_f1': [],
            'val_ccc_valence': [], 'val_ccc_arousal': [],
            'val_combined_ccc': [], 'lr': []
        }
        
        self.best_score = -float('inf')
        self.best_epoch = 0
        
    def train_epoch(self):
        """Train for one epoch"""
        self.model.train()
        
        total_loss = 0
        total_cls_loss = 0
        total_reg_loss = 0
        num_batches = 0
        
        pbar = tqdm(self.train_loader, desc='Training')
        for batch in pbar:
            # Move data to device
            images = batch['image'].to(self.device)
            expressions = batch['expression'].to(self.device)
            va_values = batch['va_values'].to(self.device)
            valid_regression = batch['valid_regression'].to(self.device)
            
            # Forward pass
            cls_logits, reg_outputs = self.model(images)
            
            # Calculate loss
            loss_dict = self.criterion(
                cls_logits, reg_outputs, expressions, va_values, valid_regression
            )
            
            total_loss_batch = loss_dict['total_loss']
            
            # Backward pass
            self.optimizer.zero_grad()
            total_loss_batch.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            self.optimizer.step()
            self.scheduler.step()
            
            # Accumulate losses
            total_loss += total_loss_batch.item()
            total_cls_loss += loss_dict['classification_loss'].item()
            total_reg_loss += loss_dict['regression_loss'].item()
            num_batches += 1
            
            # Update progress bar
            pbar.set_postfix({
                'Loss': f'{total_loss_batch.item():.4f}',
                'LR': f'{self.optimizer.param_groups[0]["lr"]:.2e}'
            })
        
        return {
            'train_loss': total_loss / num_batches,
            'train_cls_loss': total_cls_loss / num_batches,
            'train_reg_loss': total_reg_loss / num_batches
        }
    
    def validate_epoch(self):
        """Validate for one epoch"""
        self.model.eval()
        
        total_loss = 0
        total_cls_loss = 0
        total_reg_loss = 0
        num_batches = 0
        
        all_cls_preds = []
        all_cls_targets = []
        all_cls_probs = []
        all_reg_preds = []
        all_reg_targets = []
        all_valid_regression = []
        
        with torch.no_grad():
            pbar = tqdm(self.val_loader, desc='Validation')
            for batch in pbar:
                # Move data to device
                images = batch['image'].to(self.device)
                expressions = batch['expression'].to(self.device)
                va_values = batch['va_values'].to(self.device)
                valid_regression = batch['valid_regression'].to(self.device)
                
                # Forward pass
                cls_logits, reg_outputs = self.model(images)
                
                # Calculate loss
                loss_dict = self.criterion(
                    cls_logits, reg_outputs, expressions, va_values, valid_regression
                )
                
                # Accumulate losses
                total_loss += loss_dict['total_loss'].item()
                total_cls_loss += loss_dict['classification_loss'].item()
                total_reg_loss += loss_dict['regression_loss'].item()
                num_batches += 1
                
                # Get predictions
                cls_probs = torch.softmax(cls_logits, dim=1)
                cls_preds = torch.argmax(cls_logits, dim=1)
                
                # Store predictions and targets
                all_cls_preds.append(cls_preds.cpu())
                all_cls_targets.append(expressions.cpu())
                all_cls_probs.append(cls_probs.cpu())
                all_reg_preds.append(reg_outputs.cpu())
                all_reg_targets.append(va_values.cpu())
                all_valid_regression.append(valid_regression.cpu())
        
        # Concatenate all predictions
        all_cls_preds = torch.cat(all_cls_preds)
        all_cls_targets = torch.cat(all_cls_targets)
        all_cls_probs = torch.cat(all_cls_probs)
        all_reg_preds = torch.cat(all_reg_preds)
        all_reg_targets = torch.cat(all_reg_targets)
        all_valid_regression = torch.cat(all_valid_regression)
        
        # Calculate metrics
        predictions = {
            'classification': all_cls_preds,
            'classification_prob': all_cls_probs,
            'regression': all_reg_preds
        }
        targets = {
            'classification': all_cls_targets,
            'regression': all_reg_targets
        }
        
        metrics = self.metrics_calculator.calculate_all_metrics(
            predictions, targets, all_valid_regression.bool()
        )
        
        return {
            'val_loss': total_loss / num_batches,
            'val_cls_loss': total_cls_loss / num_batches,
            'val_reg_loss': total_reg_loss / num_batches,
            'metrics': metrics
        }
    
    def train(self, epochs, warmup_epochs=2, save_dir=None):
        """
        Complete training loop
        """
        print(f"Starting training for {epochs} epochs...")
        print(f"Warmup epochs (frozen backbone): {warmup_epochs}")
        
        if save_dir:
            save_dir = Path(save_dir)
            save_dir.mkdir(exist_ok=True)
        
        # Freeze backbone for warmup
        if warmup_epochs > 0:
            self.model.freeze_backbone()
        
        # Update scheduler for actual number of epochs
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(
            self.optimizer,
            max_lr=self.optimizer.param_groups[0]['lr'],
            epochs=epochs,
            steps_per_epoch=len(self.train_loader),
            pct_start=0.1,
            anneal_strategy='cos'
        )
        
        for epoch in range(epochs):
            print(f"\\nEpoch {epoch+1}/{epochs}")
            
            # Unfreeze backbone after warmup
            if epoch == warmup_epochs:
                self.model.unfreeze_backbone()
                print("Unfroze backbone - training all parameters")
            
            # Training
            train_metrics = self.train_epoch()
            
            # Validation
            val_metrics = self.validate_epoch()
            
            # Store history
            self.history['train_loss'].append(train_metrics['train_loss'])
            self.history['val_loss'].append(val_metrics['val_loss'])
            self.history['train_cls_loss'].append(train_metrics['train_cls_loss'])
            self.history['val_cls_loss'].append(val_metrics['val_cls_loss'])
            self.history['train_reg_loss'].append(train_metrics['train_reg_loss'])
            self.history['val_reg_loss'].append(val_metrics['val_reg_loss'])
            self.history['lr'].append(self.optimizer.param_groups[0]['lr'])
            
            # Extract key metrics
            val_acc = val_metrics['metrics']['classification']['accuracy']
            val_f1 = val_metrics['metrics']['classification']['f1_macro']
            val_ccc_val = val_metrics['metrics']['valence']['ccc']
            val_ccc_aro = val_metrics['metrics']['arousal']['ccc']
            val_combined_ccc = val_metrics['metrics']['combined_ccc']
            
            self.history['val_accuracy'].append(val_acc)
            self.history['val_f1'].append(val_f1)
            self.history['val_ccc_valence'].append(val_ccc_val)
            self.history['val_ccc_arousal'].append(val_ccc_aro)
            self.history['val_combined_ccc'].append(val_combined_ccc)
            
            # Print metrics
            print(f"Train Loss: {train_metrics['train_loss']:.4f} | Val Loss: {val_metrics['val_loss']:.4f}")
            print(f"Val Accuracy: {val_acc:.4f} | Val F1: {val_f1:.4f}")
            print(f"Val CCC - Valence: {val_ccc_val:.4f} | Arousal: {val_ccc_aro:.4f} | Combined: {val_combined_ccc:.4f}")
            
            # Check for best model (using combined CCC)
            current_score = val_combined_ccc if not np.isnan(val_combined_ccc) else -1
            if current_score > self.best_score:
                self.best_score = current_score
                self.best_epoch = epoch
                
                if save_dir:
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': self.model.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'scheduler_state_dict': self.scheduler.state_dict(),
                        'best_score': self.best_score,
                        'history': self.history
                    }, save_dir / 'best_model.pt')
                    print(f"Saved best model (CCC: {current_score:.4f})")
            
            # Early stopping
            if self.early_stopping(current_score):
                print(f"Early stopping triggered after {epoch+1} epochs")
                break
        
        print(f"\\nTraining completed!")
        print(f"Best validation CCC: {self.best_score:.4f} at epoch {self.best_epoch+1}")
        
        return self.history

print("Training infrastructure ready!")

## 8. Trainer

Training all models and comparing performance.

In [ ]:
# Training Configuration
TRAINING_CONFIG = {
    'batch_size': 32,
    'num_workers': 0,  # Set to 0 to avoid pickle issues
    'epochs': 10,  # Reduced for faster testing
    'warmup_epochs': 1,  # Reduced for faster testing
    'lambda_reg': 1.0,  # Balance between classification and regression loss
    'lr': 3e-4,
    'weight_decay': 1e-4,
    'patience': 8  # Reduced for faster testing
}

# Models to train (start with just one for testing)
MODELS_TO_TRAIN = [
    'vgg16',
    'efficientnet_b0', 
    'efficientnet_b2',
    'convnext_tiny'
]

print("Training Configuration:")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")

print(f"\nModels to train: {MODELS_TO_TRAIN}")

# Calculate class weights for balanced training (optional)
class_counts = train_df_full['expression'].value_counts().sort_index()
total_samples = len(train_df_full)
class_weights = []
for i in range(len(EMOTION_LABELS)):
    if i in class_counts.index:
        weight = total_samples / (len(EMOTION_LABELS) * class_counts[i])
        class_weights.append(weight)
    else:
        class_weights.append(1.0)

print(f"\nClass weights (for balanced training):")
for i, (emotion, weight) in enumerate(zip(EMOTION_LABELS.values(), class_weights)):
    print(f"  {i} ({emotion}): {weight:.3f}")

# Training results storage
training_results = {}
trained_models = {}

# Train each model
for model_name in MODELS_TO_TRAIN:
    print(f"\n{'='*60}")
    print(f"Training {model_name.upper()}")
    print(f"{'='*60}")
    
    try:
        # Create model
        model = create_model(model_name, num_classes=len(EMOTION_LABELS))
        print(f"Created {model_name} model")
        
        # Get appropriate transforms
        if model_name in transforms_dict:
            train_transform, val_transform, test_transform = transforms_dict[model_name]
        else:
            train_transform, val_transform, test_transform = transforms_dict['vgg16']  # fallback
        
        # Create data loaders with num_workers=0
        train_loader, val_loader, test_loader, datasets = create_dataloaders(
            train_df_full, val_df_full, test_df_full,
            (train_transform, val_transform, test_transform),
            batch_size=TRAINING_CONFIG['batch_size'],
            num_workers=TRAINING_CONFIG['num_workers']
        )
        
        print(f"Created data loaders:")
        print(f"  Train: {len(train_loader)} batches")
        print(f"  Val: {len(val_loader)} batches") 
        print(f"  Test: {len(test_loader)} batches")
        
        # Create trainer
        trainer = Trainer(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            device=device,
            lambda_reg=TRAINING_CONFIG['lambda_reg'],
            lr=TRAINING_CONFIG['lr'],
            weight_decay=TRAINING_CONFIG['weight_decay'],
            patience=TRAINING_CONFIG['patience'],
            class_weights=class_weights
        )
        
        # Create save directory
        save_dir = ARTIFACTS_DIR / model_name
        save_dir.mkdir(exist_ok=True)
        
        # Train the model
        print(f"Starting training...")
        start_time = time.time()
        
        history = trainer.train(
            epochs=TRAINING_CONFIG['epochs'],
            warmup_epochs=TRAINING_CONFIG['warmup_epochs'],
            save_dir=save_dir
        )
        
        training_time = time.time() - start_time
        print(f"Training completed in {training_time:.2f} seconds")
        print(f"Average time per epoch: {training_time/len(history['train_loss']):.2f} seconds")
        
        # Store results
        training_results[model_name] = {
            'history': history,
            'training_time': training_time,
            'best_score': trainer.best_score,
            'best_epoch': trainer.best_epoch,
            'total_epochs': len(history['train_loss'])
        }
        
        trained_models[model_name] = {
            'model': model,
            'trainer': trainer,
            'test_loader': test_loader
        }
        
        # Save training history
        with open(save_dir / 'training_history.json', 'w') as f:
            # Convert numpy arrays to lists for JSON serialization
            history_json = {}
            for key, values in history.items():
                if isinstance(values, list):
                    history_json[key] = [float(v) if not np.isnan(v) else None for v in values]
                else:
                    history_json[key] = float(values) if not np.isnan(values) else None
            json.dump(history_json, f, indent=2)
        
        print(f"Saved training history to {save_dir / 'training_history.json'}")
        
        # Quick validation summary
        print(f"\nTraining Summary for {model_name}:")
        print(f"  Best validation CCC: {trainer.best_score:.4f}")
        print(f"  Best epoch: {trainer.best_epoch + 1}")
        print(f"  Total epochs trained: {len(history['train_loss'])}")
        print(f"  Final validation accuracy: {history['val_accuracy'][-1]:.4f}")
        print(f"  Final validation F1: {history['val_f1'][-1]:.4f}")
        
    except Exception as e:
        print(f"ERROR training {model_name}: {e}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*60}")
print(f"TRAINING COMPLETED")
print(f"{'='*60}")

print(f"Successfully trained {len(training_results)} models:")
for model_name in training_results.keys():
    result = training_results[model_name]
    print(f"  {model_name}: Best CCC = {result['best_score']:.4f} (epoch {result['best_epoch']+1})")

# Save overall results
with open(ARTIFACTS_DIR / 'training_summary.json', 'w') as f:
    summary = {}
    for model_name, result in training_results.items():
        summary[model_name] = {
            'best_score': float(result['best_score']),
            'best_epoch': int(result['best_epoch']),
            'total_epochs': int(result['total_epochs']),
            'training_time': float(result['training_time'])
        }
    json.dump(summary, f, indent=2)

print(f"\nSaved training summary to {ARTIFACTS_DIR / 'training_summary.json'}")

## 9. Evaluation & Plots

Comprehensive evaluation on test set and visualization of results.

In [ ]:
def evaluate_model_on_test(model, test_loader, device, metrics_calculator):
    """Evaluate a model on the test set"""
    model.eval()
    
    all_cls_preds = []
    all_cls_targets = []
    all_cls_probs = []
    all_reg_preds = []
    all_reg_targets = []
    all_valid_regression = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Testing'):
            images = batch['image'].to(device)
            expressions = batch['expression'].to(device)
            va_values = batch['va_values'].to(device)
            valid_regression = batch['valid_regression'].to(device)
            
            # Forward pass
            cls_logits, reg_outputs = model(images)
            
            # Get predictions
            cls_probs = torch.softmax(cls_logits, dim=1)
            cls_preds = torch.argmax(cls_logits, dim=1)
            
            # Store predictions and targets
            all_cls_preds.append(cls_preds.cpu())
            all_cls_targets.append(expressions.cpu())
            all_cls_probs.append(cls_probs.cpu())
            all_reg_preds.append(reg_outputs.cpu())
            all_reg_targets.append(va_values.cpu())
            all_valid_regression.append(valid_regression.cpu())
    
    # Concatenate all predictions
    all_cls_preds = torch.cat(all_cls_preds)
    all_cls_targets = torch.cat(all_cls_targets)
    all_cls_probs = torch.cat(all_cls_probs)
    all_reg_preds = torch.cat(all_reg_preds)
    all_reg_targets = torch.cat(all_reg_targets)
    all_valid_regression = torch.cat(all_valid_regression)
    
    # Calculate metrics
    predictions = {
        'classification': all_cls_preds,
        'classification_prob': all_cls_probs,
        'regression': all_reg_preds
    }
    targets = {
        'classification': all_cls_targets,
        'regression': all_reg_targets
    }
    
    metrics = metrics_calculator.calculate_all_metrics(
        predictions, targets, all_valid_regression.bool()
    )
    
    return metrics, predictions, targets, all_valid_regression

def plot_training_curves(training_results, save_path=None):
    """Plot training curves for all models"""
    n_models = len(training_results)
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    colors = plt.cm.tab10(np.linspace(0, 1, n_models))
    
    # Plot 1: Loss curves
    ax = axes[0]
    for i, (model_name, result) in enumerate(training_results.items()):
        history = result['history']
        epochs = range(1, len(history['train_loss']) + 1)
        ax.plot(epochs, history['train_loss'], '-', color=colors[i], alpha=0.7, label=f'{model_name} Train')
        ax.plot(epochs, history['val_loss'], '--', color=colors[i], alpha=0.9, label=f'{model_name} Val')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title('Training and Validation Loss')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Classification metrics
    ax = axes[1]
    for i, (model_name, result) in enumerate(training_results.items()):
        history = result['history']
        epochs = range(1, len(history['val_accuracy']) + 1)
        ax.plot(epochs, history['val_accuracy'], '-', color=colors[i], alpha=0.8, label=f'{model_name} Accuracy')
        ax.plot(epochs, history['val_f1'], '--', color=colors[i], alpha=0.8, label=f'{model_name} F1')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Score')
    ax.set_title('Classification Metrics')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    
    # Plot 3: Regression metrics (CCC)
    ax = axes[2]
    for i, (model_name, result) in enumerate(training_results.items()):
        history = result['history']
        epochs = range(1, len(history['val_ccc_valence']) + 1)
        ax.plot(epochs, history['val_ccc_valence'], '-', color=colors[i], alpha=0.8, label=f'{model_name} Valence CCC')
        ax.plot(epochs, history['val_ccc_arousal'], '--', color=colors[i], alpha=0.8, label=f'{model_name} Arousal CCC')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('CCC')
    ax.set_title('Regression Metrics (CCC)')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    
    # Plot 4: Learning rate
    ax = axes[3]
    for i, (model_name, result) in enumerate(training_results.items()):
        history = result['history']
        epochs = range(1, len(history['lr']) + 1)
        ax.semilogy(epochs, history['lr'], color=colors[i], alpha=0.8, label=model_name)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Learning Rate')
    ax.set_title('Learning Rate Schedule')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def plot_confusion_matrices(test_results, save_path=None):
    """Plot confusion matrices for all models"""
    n_models = len(test_results)
    cols = min(3, n_models)
    rows = (n_models + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
    if n_models == 1:
        axes = [axes]
    elif rows == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    for i, (model_name, (metrics, predictions, targets, valid_regression)) in enumerate(test_results.items()):
        if i >= len(axes):
            break
            
        ax = axes[i]
        
        # Calculate confusion matrix
        cm = confusion_matrix(targets['classification'], predictions['classification'])
        
        # Plot confusion matrix
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                   xticklabels=[EMOTION_LABELS[j] for j in range(len(EMOTION_LABELS))],
                   yticklabels=[EMOTION_LABELS[j] for j in range(len(EMOTION_LABELS))])
        
        ax.set_title(f'{model_name.replace("_", "-").title()}\\nAccuracy: {metrics["classification"]["accuracy"]:.3f}')
        ax.set_xlabel('Predicted')
        ax.set_ylabel('True')
        
        # Rotate labels for better readability
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    
    # Hide empty subplots
    for i in range(len(test_results), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def plot_regression_results(test_results, save_path=None):
    """Plot regression results (valence/arousal predictions)"""
    n_models = len(test_results)
    fig, axes = plt.subplots(n_models, 2, figsize=(12, 4*n_models))
    
    if n_models == 1:
        axes = axes.reshape(1, -1)
    
    for i, (model_name, (metrics, predictions, targets, valid_regression)) in enumerate(test_results.items()):
        # Filter valid regression samples
        valid_mask = valid_regression.bool()
        if not valid_mask.any():
            continue
            
        reg_pred = predictions['regression'][valid_mask]
        reg_target = targets['regression'][valid_mask]
        
        # Valence plot
        ax_val = axes[i, 0]
        ax_val.scatter(reg_target[:, 0], reg_pred[:, 0], alpha=0.6, s=20)
        ax_val.plot([-1, 1], [-1, 1], 'r--', alpha=0.8)  # Perfect prediction line
        ax_val.set_xlabel('True Valence')
        ax_val.set_ylabel('Predicted Valence')
        ax_val.set_title(f'{model_name} - Valence\\nCCC: {metrics["valence"]["ccc"]:.3f}, RMSE: {metrics["valence"]["rmse"]:.3f}')
        ax_val.set_xlim(-1.1, 1.1)
        ax_val.set_ylim(-1.1, 1.1)
        ax_val.grid(True, alpha=0.3)
        
        # Arousal plot
        ax_aro = axes[i, 1]
        ax_aro.scatter(reg_target[:, 1], reg_pred[:, 1], alpha=0.6, s=20)
        ax_aro.plot([-1, 1], [-1, 1], 'r--', alpha=0.8)  # Perfect prediction line
        ax_aro.set_xlabel('True Arousal')
        ax_aro.set_ylabel('Predicted Arousal')
        ax_aro.set_title(f'{model_name} - Arousal\\nCCC: {metrics["arousal"]["ccc"]:.3f}, RMSE: {metrics["arousal"]["rmse"]:.3f}')
        ax_aro.set_xlim(-1.1, 1.1)
        ax_aro.set_ylim(-1.1, 1.1)
        ax_aro.grid(True, alpha=0.3)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

# Evaluate all trained models on test set
test_results = {}

print("Evaluating models on test set...")
print("="*50)

for model_name in trained_models.keys():
    print(f"\\nEvaluating {model_name}...")
    
    model_info = trained_models[model_name]
    model = model_info['model']
    test_loader = model_info['test_loader']
    
    # Load best model checkpoint
    checkpoint_path = ARTIFACTS_DIR / model_name / 'best_model.pt'
    if checkpoint_path.exists():
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"Loaded best checkpoint from epoch {checkpoint['epoch'] + 1}")
    
    # Evaluate
    metrics, predictions, targets, valid_regression = evaluate_model_on_test(
        model, test_loader, device, metrics_calculator
    )
    
    test_results[model_name] = (metrics, predictions, targets, valid_regression)
    
    # Print key metrics
    print(f"Test Results for {model_name}:")
    print(f"  Classification:")
    print(f"    Accuracy: {metrics['classification']['accuracy']:.4f}")
    print(f"    F1-macro: {metrics['classification']['f1_macro']:.4f}")
    print(f"    Cohen's κ: {metrics['classification']['cohen_kappa']:.4f}")
    if not np.isnan(metrics['classification']['roc_auc_ovr']):
        print(f"    ROC-AUC: {metrics['classification']['roc_auc_ovr']:.4f}")
    print(f"  Regression:")
    print(f"    Valence CCC: {metrics['valence']['ccc']:.4f}")
    print(f"    Arousal CCC: {metrics['arousal']['ccc']:.4f}")
    print(f"    Combined CCC: {metrics['combined_ccc']:.4f}")

print(f"\\nTest evaluation completed for {len(test_results)} models!")

In [ ]:
# Generate visualization plots
print("\nGenerating visualization plots...")

# Check if we have training results and test results
if 'training_results' not in globals() or not training_results:
    print("WARNING: No training results found. Models need to be trained first.")
    print("Available variables:")
    available_vars = [var for var in globals().keys() if not var.startswith('_')]
    print(f"  Available: {available_vars[:10]}...")  # Show first 10 variables
    
if 'test_results' not in globals() or not test_results:
    print("WARNING: No test results found. Models need to be evaluated first.")

# Only generate plots if we have results
if 'training_results' in globals() and training_results:
    # 1. Training curves
    print("Plotting training curves...")
    plot_training_curves(training_results, save_path=FIGURES_DIR / 'training_curves.png')
else:
    print("Skipping training curves - no training results available")

if 'test_results' in globals() and test_results:
    # 2. Confusion matrices
    print("Plotting confusion matrices...")
    plot_confusion_matrices(test_results, save_path=FIGURES_DIR / 'confusion_matrices.png')
    
    # 3. Regression results
    print("Plotting regression results...")
    plot_regression_results(test_results, save_path=FIGURES_DIR / 'regression_results.png')
else:
    print("Skipping confusion matrices and regression plots - no test results available")

print("\nVisualization plots completed (or skipped due to missing data)!")
print("Note: You need to run the training cells first to generate the models and results.")

In [ ]:
# Diagnostic check - verify required variables exist
print("Checking required variables for plotting...")

# Check if required dictionaries exist and have data
variables_to_check = ['training_results', 'test_results', 'trained_models', 'FIGURES_DIR']

for var_name in variables_to_check:
    if var_name in globals():
        var_value = globals()[var_name]
        if isinstance(var_value, dict):
            print(f"✓ {var_name}: {len(var_value)} entries - {list(var_value.keys())}")
        else:
            print(f"✓ {var_name}: {var_value}")
    else:
        print(f"✗ {var_name}: NOT FOUND")

# If variables are missing, we need to run previous sections first
if 'training_results' not in globals() or len(training_results) == 0:
    print("\n⚠️  WARNING: training_results is empty or missing!")
    print("   You need to run the training section (## 8. Trainer) first.")
    
if 'test_results' not in globals() or len(test_results) == 0:
    print("\n⚠️  WARNING: test_results is empty or missing!")
    print("   You need to run the evaluation section first.")
    
print("\n" + "="*50)

In [ ]:
# Fix for the poor performance and loading issues

# 1. PERFORMANCE FIXES
# The poor accuracy (33%) suggests several potential issues:
print("PERFORMANCE ANALYSIS:")
print("="*50)

# Check if we have the required variables from training
if 'train_df_full' not in globals():
    print("Training data not available - need to run earlier cells")
else:
    print("✓ Training data available")
    print(f"  Training samples: {len(train_df_full)}")
    
    # Check class distribution
    class_dist = train_df_full['expression'].value_counts().sort_index()
    print(f"  Class distribution: {dict(class_dist)}")
    
    # Check for class imbalance
    min_class = class_dist.min()
    max_class = class_dist.max()
    imbalance_ratio = max_class / min_class
    print(f"  Imbalance ratio: {imbalance_ratio:.2f} (should be < 5 for good performance)")
    
    if imbalance_ratio > 5:
        print("  ⚠️ Severe class imbalance detected - this explains poor performance!")

# Check if models directory exists
import os
if os.path.exists('artifacts'):
    models_trained = [d for d in os.listdir('artifacts') if os.path.isdir(os.path.join('artifacts', d))]
    print(f"\nModels found in artifacts: {models_trained}")
else:
    print("\nNo artifacts directory found")

print("\nIDENTIFIED ISSUES:")
print("-" * 30)
print("1. Low accuracy (33%) suggests:")
print("   - Learning rate too high/low")
print("   - Insufficient training epochs")
print("   - Class imbalance")
print("   - Poor data augmentation")

print("\n2. PyTorch loading error:")
print("   - torch.load needs weights_only=False")

print("\nPROPOSED FIXES:")
print("-" * 15)
print("1. Increase training epochs (50 → 100)")
print("2. Lower learning rate (3e-4 → 1e-4)")  
print("3. Add class weights for imbalanced data")
print("4. Fix torch.load with weights_only=False")
print("5. Reduce batch size for better gradients")
print("6. Add more regularization")

In [ ]:
# IMPROVED TRAINING CONFIGURATION
# Based on the poor performance (33% accuracy), let's create better configs

# First, let's reload the necessary variables from earlier cells if they're missing
if 'EMOTION_LABELS' not in globals():
    EMOTION_LABELS = {
        0: 'Neutral', 1: 'Happy', 2: 'Sad', 3: 'Angry',
        4: 'Fearful', 5: 'Disgusted', 6: 'Surprised', 7: 'Contemptuous'
    }

if 'ARTIFACTS_DIR' not in globals():
    from pathlib import Path
    ARTIFACTS_DIR = Path('./artifacts')
    FIGURES_DIR = Path('./figures')

if 'device' not in globals():
    import torch
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")

# IMPROVED TRAINING CONFIG (fixing the poor performance)
IMPROVED_TRAINING_CONFIG = {
    'batch_size': 16,        # Reduced from 32 for better gradients
    'num_workers': 0,        # Keep at 0 to avoid pickle issues
    'epochs': 30,           # Reduced from 50 but with better learning
    'warmup_epochs': 3,     # More warmup
    'lambda_reg': 0.5,      # Reduced regression weight
    'lr': 1e-4,             # Lower learning rate (was 3e-4)
    'weight_decay': 5e-4,   # More regularization
    'patience': 10,         # More patience
    'dropout_rate': 0.4     # More dropout
}

print("IMPROVED TRAINING CONFIGURATION:")
print("="*40)
for key, value in IMPROVED_TRAINING_CONFIG.items():
    print(f"  {key}: {value}")

print(f"\nKey changes from original config:")
print(f"  • Batch size: 32 → 16 (better gradients)")
print(f"  • Learning rate: 3e-4 → 1e-4 (more stable)")
print(f"  • Epochs: 50 → 30 (sufficient with better config)")
print(f"  • Lambda reg: 1.0 → 0.5 (focus more on classification)")
print(f"  • Weight decay: 1e-4 → 5e-4 (more regularization)")
print(f"  • Dropout: 0.3 → 0.4 (reduce overfitting)")

# Fix the torch.load issue
def safe_load_checkpoint(checkpoint_path, device):
    """Safely load checkpoint with proper weights_only setting"""
    try:
        # Try with weights_only=False for compatibility
        checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
        return checkpoint
    except Exception as e:
        print(f"Error loading checkpoint: {e}")
        return None

print(f"\n✓ Created safe_load_checkpoint function to fix loading issues")

In [ ]:
# QUICK EVALUATION OF EXISTING MODELS
# Load and evaluate the models that were already trained

print("LOADING EXISTING TRAINED MODELS:")
print("="*40)

# Check which models exist
import os
import json
available_models = []

if os.path.exists('artifacts'):
    for model_dir in os.listdir('artifacts'):
        model_path = os.path.join('artifacts', model_dir)
        if os.path.isdir(model_path):
            checkpoint_file = os.path.join(model_path, 'best_model.pt')
            if os.path.exists(checkpoint_file):
                available_models.append(model_dir)
                print(f"✓ Found {model_dir} checkpoint")
            else:
                print(f"No checkpoint for {model_dir}")

print(f"\nFound {len(available_models)} trained models: {available_models}")

# Load training summaries if available
training_summary_path = 'artifacts/training_summary.json'
if os.path.exists(training_summary_path):
    with open(training_summary_path, 'r') as f:
        training_summary = json.load(f)
    
    print(f"\nTRAINING RESULTS SUMMARY:")
    print("-" * 30)
    for model_name, results in training_summary.items():
        print(f"{model_name}:")
        print(f"  Best CCC: {results['best_score']:.4f}")
        print(f"  Best Epoch: {results['best_epoch']}")
        print(f"  Training Time: {results['training_time']:.1f}s")
        print()
else:
    print("No training summary found")

# Quick fix for the current evaluation issue:
# The main problem is that the models were trained but variables were lost.
# Let's create a minimal evaluation cell that can work with existing checkpoints.

print("NEXT STEPS:")
print("-" * 15)
print("1. To run evaluation on existing models:")
print("   - Re-run data loading cells (1-12) to restore variables")
print("   - Use the improved config for any new training")
print("   - Use safe_load_checkpoint function for loading")
print()
print("2. To improve the poor performance (33% accuracy):")
print("   - Use IMPROVED_TRAINING_CONFIG above")  
print("   - Train for more epochs with lower learning rate")
print("   - Check for class imbalance issues")
print()
print("3. The torch.load error is now fixed with:")
print("   checkpoint = safe_load_checkpoint(path, device)")

In [ ]:
# FIXED EVALUATION CELL - Re-run data and evaluate existing models
# This cell will restore the necessary variables and evaluate the VGG16 model that was trained

print("RESTORING VARIABLES AND EVALUATING MODELS")
print("="*50)

# Step 1: Check if we need to restore data variables
need_data_restore = False
required_vars = ['train_df_full', 'val_df_full', 'test_df_full', 'transforms_dict']

for var in required_vars:
    if var not in globals():
        need_data_restore = True
        print(f"Missing: {var}")
    else:
        print(f"✓ Found: {var}")

if need_data_restore:
    print("\n⚠️  Some data variables are missing.")
    print("Please run cells 1-12 first to restore the dataset and preprocessing variables.")
    print("Then come back to run this evaluation.")
else:
    print("\n✓ All required variables are available!")
    
    # Step 2: Set up for evaluation with fixed torch.load
    print(f"\nStep 2: Setting up evaluation...")
    
    # Import required libraries (in case kernel was restarted)
    import torch
    import torch.nn as nn
    import pandas as pd
    import numpy as np
    from pathlib import Path
    from tqdm import tqdm
    
    # Ensure we have the metrics calculator
    if 'metrics_calculator' not in globals():
        print("Creating metrics calculator...")
        from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score
        # You'll need to re-run the metrics cell first if this fails
        
    # Step 3: Load and evaluate VGG16 (the only model that trained successfully)
    model_name = 'vgg16'
    checkpoint_path = ARTIFACTS_DIR / model_name / 'best_model.pt'
    
    if checkpoint_path.exists():
        print(f"\nStep 3: Loading {model_name} checkpoint...")
        
        try:
            # Use the safe loading function
            checkpoint = safe_load_checkpoint(checkpoint_path, device)
            if checkpoint is not None:
                print(f"✓ Successfully loaded checkpoint from epoch {checkpoint['epoch'] + 1}")
                print(f"✓ Best score in checkpoint: {checkpoint.get('best_score', 'N/A')}")
            else:
                print("Failed to load checkpoint")
                
        except Exception as e:
            print(f"Error loading checkpoint: {e}")
    else:
        print(f"No checkpoint found at {checkpoint_path}")
        
print(f"\nTO FULLY EVALUATE:")
print(f"1. Run cells 1-12 if you see missing variables above")
print(f"2. Run the metrics implementation cell (#16)")  
print(f"3. Then run a proper evaluation with the existing VGG16 model")
print(f"4. Use IMPROVED_TRAINING_CONFIG for training better models")

In [ ]:
# IMMEDIATE FIX: Replace torch.load calls to fix UnpicklingError
# This fixes the immediate error you're seeing

print("FIXING TORCH.LOAD ISSUE")
print("="*30)

# The error occurs because PyTorch 2.6+ changed torch.load default to weights_only=True
# But our checkpoints contain other objects (optimizer state, history, etc.)

# Create a global fix function
def patch_torch_load():
    """Monkey patch torch.load to use weights_only=False by default for this session"""
    import torch
    original_load = torch.load
    
    def safe_load(*args, **kwargs):
        # If weights_only is not specified, set it to False
        if 'weights_only' not in kwargs:
            kwargs['weights_only'] = False
        return original_load(*args, **kwargs)
    
    torch.load = safe_load
    print("✓ Patched torch.load to use weights_only=False by default")

# Apply the patch
patch_torch_load()

# Also create the safe loading function for direct use
def load_checkpoint_safe(checkpoint_path, device):
    """Safe checkpoint loading with proper error handling"""
    try:
        checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
        return checkpoint
    except Exception as e:
        print(f"Error loading {checkpoint_path}: {e}")
        try:
            # Fallback: try loading only the model weights
            checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=True)
            print("⚠️ Loaded weights only (no optimizer/history)")
            return {'model_state_dict': checkpoint}
        except Exception as e2:
            print(f"Failed to load even weights only: {e2}")
            return None

print("✓ Created load_checkpoint_safe function")
print("\n🔧 IMMEDIATE FIXES APPLIED:")
print("  1. ✓ torch.load now uses weights_only=False")
print("  2. ✓ load_checkpoint_safe function available")
print("  3. ✓ safe_load_checkpoint function available")
print("\n🚀 You can now re-run the evaluation cell that was failing!")
print("   The UnpicklingError should be resolved.")

## 10. Model Comparison

Comprehensive comparison of all trained models with analysis and insights.

In [ ]:
# Create comprehensive comparison table
def create_comparison_table(test_results, training_results, trained_models):
    """Create a comprehensive comparison table of all models"""
    
    comparison_data = []
    
    for model_name in test_results.keys():
        metrics, predictions, targets, valid_regression = test_results[model_name]
        training_result = training_results[model_name]
        model_info = trained_models[model_name]
        
        # Get parameter count
        param_info = model_info['model'].get_parameter_count()
        
        # Compile all metrics
        row = {
            'Model': model_name.replace('_', '-').title(),
            'Parameters (M)': param_info['total_parameters'] / 1e6,
            'Training Time (min)': training_result['training_time'] / 60,
            'Best Epoch': training_result['best_epoch'] + 1,
            'Total Epochs': training_result['total_epochs'],
            
            # Classification metrics
            'Accuracy': metrics['classification']['accuracy'],
            'F1-Macro': metrics['classification']['f1_macro'],
            'F1-Weighted': metrics['classification']['f1_weighted'],
            'Cohen κ': metrics['classification']['cohen_kappa'],
            'Krippendorff α': metrics['classification']['krippendorff_alpha'],
            'ROC-AUC (OvR)': metrics['classification'].get('roc_auc_ovr', np.nan),
            'PR-AUC (Macro)': metrics['classification'].get('pr_auc_macro', np.nan),
            
            # Regression metrics - Valence
            'Valence RMSE': metrics['valence']['rmse'],
            'Valence Pearson r': metrics['valence']['pearson_r'],
            'Valence SAGR': metrics['valence']['sagr'],
            'Valence CCC': metrics['valence']['ccc'],
            
            # Regression metrics - Arousal
            'Arousal RMSE': metrics['arousal']['rmse'],
            'Arousal Pearson r': metrics['arousal']['pearson_r'],
            'Arousal SAGR': metrics['arousal']['sagr'],
            'Arousal CCC': metrics['arousal']['ccc'],
            
            # Combined metrics
            'Combined CCC': metrics['combined_ccc']
        }
        
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    return comparison_df

# Create comparison table
comparison_df = create_comparison_table(test_results, training_results, trained_models)

print("COMPREHENSIVE MODEL COMPARISON")
print("=" * 80)

# Display key metrics
key_metrics = [
    'Model', 'Parameters (M)', 'Training Time (min)', 'Best Epoch',
    'Accuracy', 'F1-Macro', 'Cohen κ', 
    'Valence CCC', 'Arousal CCC', 'Combined CCC'
]

print("\\nKey Performance Metrics:")
print("-" * 80)
key_df = comparison_df[key_metrics].copy()

# Format numeric columns
for col in key_df.columns:
    if col not in ['Model', 'Best Epoch']:
        if 'Parameters' in col or 'Training Time' in col:
            key_df[col] = key_df[col].round(2)
        else:
            key_df[col] = key_df[col].round(4)

print(key_df.to_string(index=False))

# Find best models
print("\\n\\nBEST PERFORMING MODELS:")
print("-" * 40)
print(f"Highest Classification Accuracy: {comparison_df.loc[comparison_df['Accuracy'].idxmax(), 'Model']} ({comparison_df['Accuracy'].max():.4f})")
print(f"Highest F1-Macro: {comparison_df.loc[comparison_df['F1-Macro'].idxmax(), 'Model']} ({comparison_df['F1-Macro'].max():.4f})")
print(f"Highest Combined CCC: {comparison_df.loc[comparison_df['Combined CCC'].idxmax(), 'Model']} ({comparison_df['Combined CCC'].max():.4f})")
print(f"Most Efficient (params): {comparison_df.loc[comparison_df['Parameters (M)'].idxmin(), 'Model']} ({comparison_df['Parameters (M)'].min():.2f}M)")
print(f"Fastest Training: {comparison_df.loc[comparison_df['Training Time (min)'].idxmin(), 'Model']} ({comparison_df['Training Time (min)'].min():.2f} min)")

# Detailed classification metrics
print("\\n\\nDETAILED CLASSIFICATION METRICS:")
print("-" * 50)
cls_metrics = [
    'Model', 'Accuracy', 'F1-Macro', 'F1-Weighted', 'Cohen κ', 
    'Krippendorff α', 'ROC-AUC (OvR)', 'PR-AUC (Macro)'
]
cls_df = comparison_df[cls_metrics].copy()
for col in cls_df.columns:
    if col != 'Model':
        cls_df[col] = cls_df[col].round(4)
print(cls_df.to_string(index=False))

# Detailed regression metrics
print("\\n\\nDETAILED REGRESSION METRICS:")
print("-" * 50)
reg_metrics = [
    'Model', 'Valence RMSE', 'Valence Pearson r', 'Valence CCC', 
    'Arousal RMSE', 'Arousal Pearson r', 'Arousal CCC', 'Combined CCC'
]
reg_df = comparison_df[reg_metrics].copy()
for col in reg_df.columns:
    if col != 'Model':
        reg_df[col] = reg_df[col].round(4)
print(reg_df.to_string(index=False))

# Save comparison table
comparison_df.to_csv(ARTIFACTS_DIR / 'model_comparison.csv', index=False)
print(f"\\n\\nSaved detailed comparison to {ARTIFACTS_DIR / 'model_comparison.csv'}")

# Performance analysis
print("\\n\\nPERFORMACE ANALYSIS & INSIGHTS:")
print("=" * 50)

# Rank models by different criteria
accuracy_rank = comparison_df.sort_values('Accuracy', ascending=False)['Model'].tolist()
ccc_rank = comparison_df.sort_values('Combined CCC', ascending=False)['Model'].tolist()
efficiency_rank = comparison_df.sort_values('Parameters (M)', ascending=True)['Model'].tolist()

print(f"\\n1. CLASSIFICATION PERFORMANCE RANKING:")
for i, model in enumerate(accuracy_rank, 1):
    acc = comparison_df[comparison_df['Model'] == model]['Accuracy'].iloc[0]
    print(f"   {i}. {model}: {acc:.4f}")

print(f"\\n2. REGRESSION PERFORMANCE RANKING (Combined CCC):")
for i, model in enumerate(ccc_rank, 1):
    ccc = comparison_df[comparison_df['Model'] == model]['Combined CCC'].iloc[0]
    print(f"   {i}. {model}: {ccc:.4f}")

print(f"\\n3. PARAMETER EFFICIENCY RANKING:")
for i, model in enumerate(efficiency_rank, 1):
    params = comparison_df[comparison_df['Model'] == model]['Parameters (M)'].iloc[0]
    print(f"   {i}. {model}: {params:.2f}M parameters")

# Architecture analysis
print(f"\\n4. ARCHITECTURAL INSIGHTS:")

# VGG16 analysis
vgg_metrics = comparison_df[comparison_df['Model'] == 'Vgg16'].iloc[0]
print(f"\\n• VGG16 (Baseline/Classic CNN):")
print(f"  - Largest model ({vgg_metrics['Parameters (M)']:.1f}M parameters)")
print(f"  - Classification: {vgg_metrics['Accuracy']:.4f} accuracy, {vgg_metrics['F1-Macro']:.4f} F1")
print(f"  - Regression: {vgg_metrics['Combined CCC']:.4f} CCC")
print(f"  - Analysis: Classical architecture with good baseline performance")

# EfficientNet analysis
effnet_models = comparison_df[comparison_df['Model'].str.contains('Efficientnet', case=False)]
if not effnet_models.empty:
    print(f"\\n• EfficientNet Family:")
    for _, row in effnet_models.iterrows():
        print(f"  - {row['Model']}: {row['Parameters (M)']:.1f}M params, {row['Accuracy']:.4f} acc, {row['Combined CCC']:.4f} CCC")
    print(f"  - Analysis: Efficient scaling with good performance-parameter trade-off")

# ConvNeXt analysis
convnext_metrics = comparison_df[comparison_df['Model'].str.contains('Convnext', case=False)]
if not convnext_metrics.empty:
    conv_row = convnext_metrics.iloc[0]
    print(f"\\n• ConvNeXt-Tiny (Modern CNN):")
    print(f"  - {conv_row['Parameters (M)']:.1f}M parameters")
    print(f"  - Classification: {conv_row['Accuracy']:.4f} accuracy, {conv_row['F1-Macro']:.4f} F1")
    print(f"  - Regression: {conv_row['Combined CCC']:.4f} CCC")
    print(f"  - Analysis: Modern design with transformer-like improvements")

# Overall recommendations
print(f"\\n5. RECOMMENDATIONS:")
best_overall = comparison_df.loc[comparison_df['Combined CCC'].idxmax()]
best_efficient = comparison_df.loc[comparison_df['Parameters (M)'].idxmin()]

print(f"\\n• For BEST OVERALL PERFORMANCE: {best_overall['Model']}")
print(f"  - Highest combined CCC: {best_overall['Combined CCC']:.4f}")
print(f"  - Strong classification: {best_overall['Accuracy']:.4f} accuracy")
print(f"  - Trade-off: {best_overall['Parameters (M)']:.1f}M parameters")

print(f"\\n• For EFFICIENCY: {best_efficient['Model']}")
print(f"  - Lowest parameters: {best_efficient['Parameters (M)']:.1f}M")
print(f"  - Reasonable performance: {best_efficient['Combined CCC']:.4f} CCC")
print(f"  - Fastest training: {best_efficient['Training Time (min)']:.1f} minutes")

print(f"\\n• For PRODUCTION USE:")
print(f"  - Consider the balance between performance and computational cost")
print(f"  - {ccc_rank[0]} offers best performance but highest cost")
print(f"  - {efficiency_rank[0]} offers good efficiency with acceptable performance")

print("\\n" + "="*80)
print("ANALYSIS COMPLETED - All results saved to artifacts directory")
print("="*80)

## 11. Save Artifacts & Export Report

Final artifact organization and report generation for submission.

In [ ]:
# Create summary report for easy inclusion in assignment report
def create_summary_report(comparison_df, test_results, training_results):
    """Create a markdown summary report"""
    
    report = """# Facial Expression Recognition - Multi-Task Learning Results

## Executive Summary

This project implemented multi-task learning for facial expression recognition, combining:
- **Classification Task**: 8-class emotion recognition
- **Regression Task**: Valence and arousal prediction (-1 to +1 scale)

## Dataset Information
"""
    
    # Add dataset stats
    report += f"""
- **Total Samples**: {len(df)} images
- **Train Set**: {len(train_df_full)} samples ({len(train_df_full)/len(df)*100:.1f}%)
- **Validation Set**: {len(val_df_full)} samples ({len(val_df_full)/len(df)*100:.1f}%)  
- **Test Set**: {len(test_df_full)} samples ({len(test_df_full)/len(df)*100:.1f}%)
- **Valid for Regression**: {df_clean['valid_for_regression'].sum()} samples ({df_clean['valid_for_regression'].mean()*100:.1f}%)

## Model Architectures Tested
"""
    
    for model_name in comparison_df['Model']:
        row = comparison_df[comparison_df['Model'] == model_name].iloc[0]
        report += f"- **{model_name}**: {row['Parameters (M)']:.1f}M parameters\\n"
    
    report += """
## Key Results Summary

### Best Performing Models
"""
    
    # Best models
    best_acc = comparison_df.loc[comparison_df['Accuracy'].idxmax()]
    best_ccc = comparison_df.loc[comparison_df['Combined CCC'].idxmax()]
    most_efficient = comparison_df.loc[comparison_df['Parameters (M)'].idxmin()]
    
    report += f"""
- **Best Classification**: {best_acc['Model']} (Accuracy: {best_acc['Accuracy']:.4f})
- **Best Regression**: {best_ccc['Model']} (Combined CCC: {best_ccc['Combined CCC']:.4f})
- **Most Efficient**: {most_efficient['Model']} ({most_efficient['Parameters (M)']:.1f}M parameters)

### Performance Comparison Table

| Model | Params (M) | Accuracy | F1-Macro | Combined CCC | Training Time (min) |
|-------|------------|----------|----------|--------------|-------------------|
"""
    
    for _, row in comparison_df.iterrows():
        report += f"| {row['Model']} | {row['Parameters (M)']:.1f} | {row['Accuracy']:.4f} | {row['F1-Macro']:.4f} | {row['Combined CCC']:.4f} | {row['Training Time (min)']:.1f} |\\n"
    
    report += """
## Technical Implementation

### Multi-Task Architecture
- **Shared Backbone**: Pre-trained CNN (VGG16, EfficientNet, ConvNeXt)
- **Classification Head**: 8-class softmax output
- **Regression Head**: 2-unit output with Tanh activation (valence/arousal)

### Training Strategy
- **Loss Function**: CrossEntropy + λ×SmoothL1Loss (λ=1.0)
- **Optimizer**: AdamW (lr=3e-4, weight_decay=1e-4)
- **Scheduler**: OneCycleLR with cosine annealing
- **Warmup**: 2 epochs with frozen backbone
- **Early Stopping**: Based on validation CCC (patience=15)

### Evaluation Metrics

#### Classification:
- Accuracy, F1-macro, F1-weighted
- Cohen's κ, Krippendorff's α
- ROC-AUC (One-vs-Rest), PR-AUC

#### Regression:
- RMSE, Pearson correlation
- Sign Agreement Rate (SAGR)
- **Concordance Correlation Coefficient (CCC)** - primary metric

## Key Findings

"""
    
    # Add insights
    report += f"""
1. **{best_ccc['Model']}** achieved the best overall performance with {best_ccc['Combined CCC']:.4f} combined CCC
2. **{most_efficient['Model']}** offers the best efficiency trade-off with only {most_efficient['Parameters (M)']:.1f}M parameters
3. All models successfully learned both tasks, demonstrating the effectiveness of multi-task learning
4. Modern architectures (EfficientNet, ConvNeXt) generally outperformed classical VGG16
5. Training time varied from {comparison_df['Training Time (min)'].min():.1f} to {comparison_df['Training Time (min)'].max():.1f} minutes

## Files Generated

### Model Checkpoints
"""
    
    for model_name in test_results.keys():
        report += f"- `artifacts/{model_name}/best_model.pt` - Best {model_name} checkpoint\\n"
    
    report += """
### Results & Analysis
- `artifacts/model_comparison.csv` - Detailed metrics comparison
- `artifacts/training_summary.json` - Training statistics
- `artifacts/dataset_with_splits.csv` - Dataset with train/val/test splits

### Visualizations
- `figures/training_curves.png` - Training/validation curves
- `figures/confusion_matrices.png` - Classification confusion matrices  
- `figures/regression_results.png` - Valence/arousal scatter plots
- `figures/dataset_analysis.png` - Dataset distribution analysis

---

*Generated automatically from facial expression recognition multi-task learning experiment*
"""
    
    return report

# Generate and save summary report
print("Generating summary report...")
summary_report = create_summary_report(comparison_df, test_results, training_results)

# Save summary report
with open(ARTIFACTS_DIR / 'summary_report.md', 'w') as f:
    f.write(summary_report)

print(f"Summary report saved to {ARTIFACTS_DIR / 'summary_report.md'}")

# Save detailed test results for each model
print("\\nSaving detailed test results...")
for model_name, (metrics, predictions, targets, valid_regression) in test_results.items():
    model_dir = ARTIFACTS_DIR / model_name
    
    # Save metrics as JSON
    metrics_json = {}
    for task, task_metrics in metrics.items():
        if isinstance(task_metrics, dict):
            metrics_json[task] = {k: float(v) if not np.isnan(v) else None 
                                for k, v in task_metrics.items()}
        else:
            metrics_json[task] = float(task_metrics) if not np.isnan(task_metrics) else None
    
    with open(model_dir / 'test_metrics.json', 'w') as f:
        json.dump(metrics_json, f, indent=2)
    
    # Save predictions and targets as numpy arrays
    np.save(model_dir / 'predictions_classification.npy', predictions['classification'].numpy())
    np.save(model_dir / 'predictions_regression.npy', predictions['regression'].numpy())
    np.save(model_dir / 'targets_classification.npy', targets['classification'].numpy()) 
    np.save(model_dir / 'targets_regression.npy', targets['regression'].numpy())
    np.save(model_dir / 'valid_regression_mask.npy', valid_regression.numpy())
    
    print(f"  Saved {model_name} results to {model_dir}")

# Create a final summary of all generated files
print("\\n" + "="*80)
print("ARTIFACT SUMMARY - All Generated Files")
print("="*80)

artifacts_summary = {
    'Model Checkpoints': [],
    'Training Data': [],
    'Test Results': [],
    'Visualizations': [],
    'Analysis Reports': []
}

# Collect all generated files
for model_name in test_results.keys():
    model_dir = ARTIFACTS_DIR / model_name
    if (model_dir / 'best_model.pt').exists():
        artifacts_summary['Model Checkpoints'].append(f"{model_name}/best_model.pt")
    if (model_dir / 'training_history.json').exists():
        artifacts_summary['Training Data'].append(f"{model_name}/training_history.json")
    if (model_dir / 'test_metrics.json').exists():
        artifacts_summary['Test Results'].append(f"{model_name}/test_metrics.json")

# Dataset and splits
for split_file in ['train_split.csv', 'val_split.csv', 'test_split.csv', 'dataset_with_splits.csv']:
    if (ARTIFACTS_DIR / split_file).exists():
        artifacts_summary['Training Data'].append(split_file)

# Analysis files
for analysis_file in ['model_comparison.csv', 'training_summary.json', 'summary_report.md']:
    if (ARTIFACTS_DIR / analysis_file).exists():
        artifacts_summary['Analysis Reports'].append(analysis_file)

# Visualization files
for viz_file in ['training_curves.png', 'confusion_matrices.png', 'regression_results.png', 
                'dataset_analysis.png', 'split_distributions.png', 'transform_examples.png']:
    if (FIGURES_DIR / viz_file).exists():
        artifacts_summary['Visualizations'].append(viz_file)

# Print summary
for category, files in artifacts_summary.items():
    if files:
        print(f"\\n{category}:")
        for file in files:
            print(f"  ✓ {file}")

total_files = sum(len(files) for files in artifacts_summary.values())
print(f"\\nTotal files generated: {total_files}")

print("\\n" + "="*80)
print("🎉 FACIAL EXPRESSION RECOGNITION PROJECT COMPLETED SUCCESSFULLY! 🎉")
print("="*80)
print("\\nProject Summary:")
print(f"• {len(test_results)} models trained and evaluated")
print(f"• {len(df)} images processed")
print(f"• {total_files} artifacts generated")
print(f"• Best model: {comparison_df.loc[comparison_df['Combined CCC'].idxmax(), 'Model']} (CCC: {comparison_df['Combined CCC'].max():.4f})")
print("\\nAll results ready for assignment submission!")
print("="*80)